ゲームタイトル：サイコロウォーズ（仮称）
ゲーム概要
『サイコロウォーズ』は、2人対戦型の陣取りボードゲームである。各プレイヤーは1個ずつのサイコロを持ち、交互にサイコロを盤面上で転がすことで、通ったマスを自分の色に塗り替えていく。最終的に、盤面上で自分の色のマスが多いプレイヤーが勝者となる。

プレイ人数
2人対戦（1 vs 1）

盤面仕様
	• 盤面は 6×6マスの正方形グリッド

基本ルール
初期設定
	• プレイヤー1（先手）：白
	• プレイヤー2（後手）：黒
	• 各プレイヤーは自分のサイコロを任意の初期位置に置く
サイコロの転がし方
	• はじめのサイコロの一番上の面の数字が、そのターンにサイコロを転がせる回数となる
例：上面が「4」→ 4回転がすことができる
	• 転がす方向はプレイヤーが自由に選択可能（上下左右の4方向）
	• 転がすたびにサイコロの上下左右の面は当然変化する
	• 移動終了時に上面に出ている数字が、次のターンで移動できる数となる
色の塗り方
	• 自分のサイコロが通過したマスは、自分の色に変化する
	• すでに相手の色になっているマスも、通過すれば上書き塗り替え可能
	• 自分の最後移動した後に敵のさいころと位置がかぶった場合は敵のさいころをキルして回りの8マス(周囲のマス)を自分の色に塗り替えることができる。
	• キルされたサイコロはランダムな位置にキルされた位置以外のランダムな場所にランダムな角度でスポーンする。

勝敗条件
	• 各20ターンをプレイして盤面上の自分の色のマスの数が多い方が勝利

特徴
	• サイコロの目に応じて戦略的なルート設計が求められる
	• 相手の塗ったマスを奪い返す、陣取り型の頭脳戦
  •転がす方向と数字の管理により、思考性と運のバランスが特徴的なゲーム

In [1]:
!pip install triton==2.0.0
!pip install numpy

INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.4/96.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 56.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 34.2 MB/s eta 0

In [2]:





import numpy as np
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ExponentialLR
from collections import deque
import time
import os
import math
# 進捗表示のために tqdm をインポート
from tqdm import tqdm

# --- 保存先ディレクトリを指定 ---
# 注意: 実行環境に合わせてパスを適切に変更してください
save_dir = r"E:\OneDrive - 学校法人　永守学園\個人ファイル\Python to UnrealEngine5\DiceWars_EvenDeeper_Strategic" # 保存先名を変更
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# --- サイコロクラス（変更なし）---
class Dice:
    def __init__(self, pos):
        top = random.randint(1, 6)
        bottom = 7 - top
        candidates = [n for n in range(1, 7) if n not in [top, bottom]]
        left = random.choice(candidates)
        right = 7 - left
        # Ensure candidates has enough elements before removing left and choosing front
        if left in candidates:
            candidates.remove(left)
        if not candidates: # Handle edge case if somehow candidates becomes empty (shouldn't happen with standard dice)
             # Fallback: Find remaining possible faces
             all_faces = set(range(1, 7))
             used_faces = {top, bottom, left, right}
             candidates = list(all_faces - used_faces)
             if not candidates: # Should be impossible if logic is correct
                  raise ValueError("Cannot determine front/back faces")

        front = random.choice(candidates)
        back = 7 - front
        self.orientation = (top, bottom, left, right, front, back)
        self.pos = pos # (row, col)

    @property
    def top(self):
        return self.orientation[0]

    def roll(self, direction):
        top, bottom, left, right, front, back = self.orientation
        if direction == 'up': # 前に転がす (奥が新しい上面)
            self.orientation = (front, back, left, right, bottom, top)
        elif direction == 'down': # 手前に転がす (手前が新しい上面)
            self.orientation = (back, front, left, right, top, bottom)
        elif direction == 'right': # 右に転がす (左が新しい上面)
            self.orientation = (left, right, bottom, top, front, back)
        elif direction == 'left': # 左に転がす (右が新しい上面)
            self.orientation = (right, left, top, bottom, front, back)
        else:
            raise ValueError("無効な方向です")

# --- ゲーム環境クラス（報酬設計を変更）---
class PsychoDiceWarsEnv:
    def __init__(self, board_size=6, max_turns=40):
        self.board_size = board_size
        self.max_turns = max_turns
        self.reset()

    def reset(self):
        self.board = np.zeros((self.board_size, self.board_size), dtype=int)
        self.turn_count = 0
        self.current_player = 1 # Player 1 starts
        self.dice1 = None # Initialize as None
        self.dice2 = None # Initialize as None
        self.moves_remaining = {1: 0, 2: 0}
        self.done = False

        # Ensure dice start in different positions
        while True:
            pos1 = (random.randint(0, self.board_size - 1), random.randint(0, self.board_size - 1))
            pos2 = (random.randint(0, self.board_size - 1), random.randint(0, self.board_size - 1))
            if pos1 != pos2:
                break
        try:
            self.dice1 = Dice(pos1)
            self.dice2 = Dice(pos2)
            self.board[pos1] = 1
            self.board[pos2] = 2
            self.moves_remaining = {1: self.dice1.top, 2: self.dice2.top}
        except Exception as e:
            print(f"Error initializing dice: {e}. Retrying reset.")
            # Avoid infinite recursion if reset keeps failing
            # Consider adding a counter or specific error handling
            return self.reset() # Recursive call might be risky, iterative approach preferred if issues arise

        self.done = False
        return self.get_state()

    def get_state(self):
        # Ensure board is float32
        board_flat = self.board.flatten().astype(np.float32)

        # Handle cases where dice might be None (e.g., during reset issues or kill)
        dice1_exists = self.dice1 is not None
        dice2_exists = self.dice2 is not None

        # Represent player perspective: "self" is always the current player
        if self.current_player == 1:
            pos_self = np.array(self.dice1.pos, dtype=np.float32) if dice1_exists else np.array([-1, -1], dtype=np.float32)
            pos_opp = np.array(self.dice2.pos, dtype=np.float32) if dice2_exists else np.array([-1, -1], dtype=np.float32)
            moves = np.array([self.moves_remaining.get(1, 0)], dtype=np.float32)
            dice_top_self = np.array([self.dice1.top], dtype=np.float32) if dice1_exists else np.array([0], dtype=np.float32)
            dice_top_opp = np.array([self.dice2.top], dtype=np.float32) if dice2_exists else np.array([0], dtype=np.float32)
        else: # Current player is 2
            pos_self = np.array(self.dice2.pos, dtype=np.float32) if dice2_exists else np.array([-1, -1], dtype=np.float32)
            pos_opp = np.array(self.dice1.pos, dtype=np.float32) if dice1_exists else np.array([-1, -1], dtype=np.float32)
            moves = np.array([self.moves_remaining.get(2, 0)], dtype=np.float32)
            dice_top_self = np.array([self.dice2.top], dtype=np.float32) if dice2_exists else np.array([0], dtype=np.float32)
            dice_top_opp = np.array([self.dice1.top], dtype=np.float32) if dice1_exists else np.array([0], dtype=np.float32)

        turn_normalized = np.array([self.turn_count / self.max_turns], dtype=np.float32)

        # Concatenate all features
        state_parts = [board_flat, pos_self, pos_opp, moves, dice_top_self, dice_top_opp, turn_normalized]
        state = np.concatenate(state_parts)

        # Final check for expected dimension (6*6 + 2 + 2 + 1 + 1 + 1 + 1 = 44)
        expected_dim = self.board_size * self.board_size + 2 + 2 + 1 + 1 + 1 + 1
        if state.shape[0] != expected_dim:
             print(f"Warning: State dimension mismatch! Expected {expected_dim}, Got {state.shape[0]}")
             # Attempt to fix or raise error depending on severity
             # This might indicate an issue in how state components are derived or concatenated
             # For now, let's pad/truncate, but this needs investigation
             if state.shape[0] < expected_dim:
                  state = np.pad(state, (0, expected_dim - state.shape[0]), 'constant', constant_values=0.0)
             else:
                  state = state[:expected_dim]


        # Ensure no NaN/Inf creep in
        state = np.nan_to_num(state, nan=0.0, posinf=0.0, neginf=0.0)

        return state.astype(np.float32) # Ensure final type

    def print_board(self):
        symbol = {0: '.', 1: 'W', 2: 'B'}
        dice1_pos = self.dice1.pos if hasattr(self, 'dice1') and self.dice1 else None
        dice2_pos = self.dice2.pos if hasattr(self, 'dice2') and self.dice2 else None
        dice1_top = self.dice1.top if hasattr(self, 'dice1') and self.dice1 else '?'
        dice2_top = self.dice2.top if hasattr(self, 'dice2') and self.dice2 else '?'

        print("-" * (self.board_size * 3))
        for r in range(self.board_size):
            row_str = ""
            for c in range(self.board_size):
                pos = (r, c)
                if pos == dice1_pos:
                    row_str += f"W{dice1_top} "
                elif pos == dice2_pos:
                    row_str += f"B{dice2_top} "
                else:
                    # Ensure board values are valid keys for symbol dict
                    board_val = self.board[r, c]
                    display_char = symbol.get(board_val, '?') # Use '?' for unexpected values
                    row_str += display_char + "  "
            print(row_str)
        moves_left = self.moves_remaining.get(self.current_player, 0)
        player_char = 'W' if self.current_player == 1 else 'B'
        print(f"Turn: {self.turn_count}/{self.max_turns}, Player: {player_char}, Moves Left: {moves_left}")
        # Optionally show dice orientations for debugging
        # if self.dice1: print(f"Dice1 (W): {self.dice1.orientation} @ {self.dice1.pos}")
        # if self.dice2: print(f"Dice2 (B): {self.dice2.orientation} @ {self.dice2.pos}")
        print("-" * (self.board_size * 3))

    def valid_move(self, pos, direction):
        r, c = pos
        dr, dc = 0, 0
        if direction == 'up': dr = -1
        elif direction == 'down': dr = 1
        elif direction == 'left': dc = -1
        elif direction == 'right': dc = 1

        next_r, next_c = r + dr, c + dc
        is_valid_pos = 0 <= next_r < self.board_size and 0 <= next_c < self.board_size
        return is_valid_pos, (next_r, next_c)

    def step(self, action):
        # Safety check for dice initialization
        if not hasattr(self, 'dice1') or not self.dice1 or not hasattr(self, 'dice2') or not self.dice2:
            # This state should ideally not be reached in normal play after reset
            # If it happens, it indicates a potential issue in reset or kill logic
            print("Critical Warning: Dice objects not properly initialized during step. Returning terminal state.")
            # Return a state indicating an error or end game immediately
            # Returning current state might lead to infinite loops if state is invalid
            dummy_state = np.zeros(44, dtype=np.float32) # Match state dimension
            return dummy_state, -100, True, {'error': 'Dice not initialized during step'} # High penalty, end episode

        actions = ['up', 'down', 'left', 'right']
        chosen_dir = actions[action] # Action is an index 0-3

        # Identify current player's dice and opponent
        if self.current_player == 1:
            dice = self.dice1
            player_color = 1
            enemy_dice = self.dice2
            enemy_player = 2
        else: # current_player == 2
            dice = self.dice2
            player_color = 2
            enemy_dice = self.dice1
            enemy_player = 1

        # --- Reward Calculation Logic ---
        reward = 0.0 # Initialize step reward
        info = {'battle_result': 'none', 'turn_ended': False, 'final_score': None}
        original_pos = dice.pos # Store position before rolling

        # 1. Perform Roll and Check Movement Validity
        try:
            dice.roll(chosen_dir) # Dice always rolls, changing its top face
        except ValueError as e:
             print(f"Error during dice roll: {e}. Direction: {chosen_dir}. Action: {action}")
             # Handle error state - maybe assign penalty and end turn?
             reward -= 10 # Severe penalty for invalid action logic
             self.moves_remaining[self.current_player] = 0 # Force end of moves
             info['error'] = 'Invalid roll direction'
             # Proceed to turn end logic below

        valid_grid, next_pos_candidate = self.valid_move(original_pos, chosen_dir) # Use original pos for move check

        if valid_grid:
            target_tile_owner = self.board[next_pos_candidate] # Check owner BEFORE moving
            dice.pos = next_pos_candidate # Update dice position
            self.board[dice.pos] = player_color # Update board AFTER checking owner

            # --- Movement Reward based on target tile ---
            if target_tile_owner == 0:
                move_reward = 0.1  # Painting empty space
            elif target_tile_owner == enemy_player:
                move_reward = 0.3  # Taking enemy space (higher value)
            else: # target_tile_owner == player_color
                move_reward = 0.05 # Moving over own space (slight incentive)
            reward += move_reward
        else:
            # Invalid Move (hit wall/edge) - Penalty
            reward -= 0.5
            # Dice position does not change, but orientation (top face) did change due to roll

        # 2. Decrement Moves and Check for Turn End
        if self.moves_remaining[self.current_player] > 0:
             self.moves_remaining[self.current_player] -= 1
        else:
             # This case should ideally not happen if moves are managed correctly
             print(f"Warning: Player {self.current_player} attempted move with 0 moves remaining.")


        turn_ends_this_step = (self.moves_remaining[self.current_player] <= 0)

        if turn_ends_this_step:
            info['turn_ended'] = True
            current_dice_at_turn_end = dice # The dice that just finished its moves

            # --- End-of-Turn Rewards / Penalties ---
            # a) Check for Kill (highest priority reward/outcome)
            kill_occurred = False
            if enemy_dice and current_dice_at_turn_end.pos == enemy_dice.pos:
                try:
                    self._kill(enemy_player, current_dice_at_turn_end.pos)
                    reward += 5.0  # Significant reward for kill
                    info['battle_result'] = 'kill'
                    kill_occurred = True
                    # After a kill, the enemy dice object (enemy_dice) might be replaced,
                    # so subsequent checks involving enemy_dice need care or re-evaluation.
                    # Update local enemy_dice variable if necessary
                    enemy_dice = self.dice1 if player_color == 2 else self.dice2

                except Exception as e:
                     print(f"Error during kill processing: {e}")
                     reward -= 10 # Penalize if kill logic fails
                     info['error'] = 'Kill processing failed'


            # b) If NO kill occurred, evaluate other end-of-turn conditions
            if not kill_occurred:
                # i) Top Face Bonus: Did the player end with 6 on top?
                if current_dice_at_turn_end.top == 6:
                    reward += 0.5 # Bonus for maximizing next turn's potential

                # ii) Vulnerability Penalty: Did the player end adjacent to the opponent?
                if enemy_dice: # Check if opponent still exists
                    dx = abs(current_dice_at_turn_end.pos[0] - enemy_dice.pos[0])
                    dy = abs(current_dice_at_turn_end.pos[1] - enemy_dice.pos[1])
                    manhattan_distance = dx + dy
                    if manhattan_distance == 1:
                        reward -= 1.0 # Penalty for ending turn in a vulnerable adjacent spot

            # --- Switch Player and Reset Moves for Next Player ---
            next_player = 2 if self.current_player == 1 else 1
            next_player_dice = self.dice2 if next_player == 2 else self.dice1

            if next_player_dice: # Make sure the next player's dice exists
                 self.moves_remaining[next_player] = next_player_dice.top
            else:
                 # This could happen if the next player was just killed and hasn't respawned yet?
                 # Should be handled by _kill setting moves, but add safety.
                 print(f"Warning: Next player ({next_player}) dice is missing at turn switch.")
                 self.moves_remaining[next_player] = 0 # Or maybe assign random 1-6? Needs decision.

            # Actual player switch
            if self.current_player == 1:
                self.current_player = 2
            else: # Player 2 finished
                self.current_player = 1
                self.turn_count += 1 # Increment turn count only after player 2 finishes

        # 3. Check for Game End (Max Turns)
        if not self.done and self.turn_count >= self.max_turns:
            self.done = True
            score1 = np.sum(self.board == 1)
            score2 = np.sum(self.board == 2)
            info['final_score'] = (score1, score2)

            # Calculate final reward based on game outcome (for the agent, Player 1)
            if score1 > score2: # Agent Wins
                final_reward = (score1 - score2) * 0.1 + 15.0 # Win bonus + score diff bonus
            elif score2 > score1: # Agent Loses
                final_reward = (score1 - score2) * 0.1 - 15.0 # Loss penalty + score diff penalty (negative)
            else: # Draw
                final_reward = 0.0
            reward += final_reward # Add to the reward of the final step

        # --- Prepare return values ---
        next_state_raw = self.get_state() # Get state for the *next* player

        # Determine the reward to return *for the agent (Player 1)*
        # If Player 1 just moved, return the calculated reward.
        # If Player 2 just moved (and it's now Player 1's turn), Player 1 gets 0 reward for Player 2's action.
        # The 'reward' variable holds the reward pertinent to the player *who just acted*.
        agent_reward = reward if player_color == 1 else 0.0

        # Safety check for invalid reward values
        if np.isnan(agent_reward) or np.isinf(agent_reward):
            print(f"Warning: Invalid agent_reward ({agent_reward}) generated. Clamping to 0.")
            agent_reward = 0.0

        # Safety check for invalid state values (already done in get_state, but double check)
        if np.isnan(next_state_raw).any() or np.isinf(next_state_raw).any():
             print(f"Critical Warning: NaN/Inf detected in next_state_raw before returning from step. State: {next_state_raw}")
             next_state_raw = np.nan_to_num(next_state_raw, nan=0.0, posinf=0.0, neginf=0.0)


        return next_state_raw.astype(np.float32), float(agent_reward), self.done, info

    # Kill method with slight refinement for respawn location
    def _kill(self, player_to_kill, killer_pos):
        # player_to_kill: 1 or 2 (the one being removed)
        # killer_pos: tuple (r, c) where the kill happened
        killed_player_color = player_to_kill
        killer_player_color = 3 - killed_player_color # The other player's color

        # --- 1. Conquer Area ---
        r_kill, c_kill = killer_pos
        for dr in [-1, 0, 1]:
            for dc in [-1, 0, 1]:
                nr, nc = r_kill + dr, c_kill + dc
                if 0 <= nr < self.board_size and 0 <= nc < self.board_size:
                    self.board[nr, nc] = killer_player_color # Paint 3x3 area

        # --- 2. Find Respawn Location ---
        respawn_attempts = 0
        max_attempts = self.board_size * self.board_size * 2 # Increase attempts just in case

        # Identify the killer's dice to avoid spawning on it
        killer_dice = self.dice1 if killer_player_color == 1 else self.dice2
        killer_dice_pos = killer_dice.pos if killer_dice else None # Killer might be None temporarily? Unlikely.

        new_pos = None
        while respawn_attempts < max_attempts:
            new_r = random.randint(0, self.board_size - 1)
            new_c = random.randint(0, self.board_size - 1)
            candidate_pos = (new_r, new_c)

            # Check if the spot is NOT the killer's current position
            is_safe_spot = (candidate_pos != killer_dice_pos)

            if is_safe_spot:
                new_pos = candidate_pos
                break
            respawn_attempts += 1
        else:
            # Fallback if no "safe" spot found (highly unlikely unless board is tiny)
            print(f"Warning: Could not find a respawn location different from the killer for player {player_to_kill}. Forcing respawn.")
            # Just pick any random spot, even if it's the killer's spot (will be overwritten)
            new_r = random.randint(0, self.board_size - 1)
            new_c = random.randint(0, self.board_size - 1)
            new_pos = (new_r, new_c)

        # --- 3. Respawn Dice ---
        if player_to_kill == 1:
            self.dice1 = Dice(new_pos)
            self.board[new_pos] = 1 # Place new dice on board
            self.moves_remaining[1] = self.dice1.top # Reset moves for killed player
            # print(f"Player 1 Respawned at {new_pos} with top {self.dice1.top}") # Debug
        else: # player_to_kill == 2
            self.dice2 = Dice(new_pos)
            self.board[new_pos] = 2 # Place new dice on board
            self.moves_remaining[2] = self.dice2.top # Reset moves for killed player
            # print(f"Player 2 Respawned at {new_pos} with top {self.dice2.top}") # Debug

# --- DQNネットワーク（変更なし）---
class DQN(nn.Module):
    # Input dimension should match the state size from env.get_state()
    def __init__(self, input_dim=44, output_dim=4): # Default 44 for 6x6 board
        super(DQN, self).__init__()
        # Increased complexity slightly
        self.net = nn.Sequential(
            nn.Linear(input_dim, 256), nn.ReLU(),
            nn.Linear(256, 512), nn.ReLU(), # Wider layer
            # nn.Dropout(0.1), # Optional dropout
            nn.Linear(512, 256), nn.ReLU(),
            nn.Linear(256, 128), nn.ReLU(),
            # nn.Dropout(0.1), # Optional dropout
            nn.Linear(128, 64), nn.ReLU(),
            nn.Linear(64, output_dim)
        )
        # Optional: Initialization strategy
        # for m in self.net.modules():
        #     if isinstance(m, nn.Linear):
        #         nn.init.kaiming_normal_(m.weight, mode='fan_in', nonlinearity='relu')
        #         if m.bias is not None:
        #             nn.init.constant_(m.bias, 0)


    def forward(self, x):
        # Ensure input is float32
        if x.dtype != torch.float32:
            x = x.float()
        # Check for NaN/Inf in input tensor
        if torch.isnan(x).any() or torch.isinf(x).any():
             print("Warning: NaN/Inf detected in input tensor to DQN forward pass. Clamping.")
             x = torch.nan_to_num(x, nan=0.0, posinf=1e6, neginf=-1e6) # Clamp large values too
        return self.net(x)


# --- DQNエージェント（変更なし - LRスケジューラ含む）---
class DQNAgent:
    def __init__(self, input_dim=44, output_dim=4,
                 lr=5e-5, gamma=0.99, epsilon=1.0, epsilon_min=0.05,
                 epsilon_decay=0.999995, buffer_size=1_000_000, batch_size=256,
                 target_update_freq=500, lr_scheduler_gamma=0.9995):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"Using device: {self.device}")
        self.input_dim = input_dim
        self.output_dim = output_dim

        self.model = DQN(input_dim, output_dim).to(self.device)
        self.target_model = DQN(input_dim, output_dim).to(self.device)
        self.target_model.load_state_dict(self.model.state_dict())
        self.target_model.eval() # Target network is only for inference

        self.optimizer = optim.AdamW(self.model.parameters(), lr=lr, amsgrad=True) # AdamW often performs well
        self.scheduler = ExponentialLR(self.optimizer, gamma=lr_scheduler_gamma)

        self.gamma = gamma          # Discount factor for future rewards
        self.epsilon = epsilon      # Initial exploration rate
        self.epsilon_min = epsilon_min # Minimum exploration rate
        self.epsilon_decay = epsilon_decay # Rate at which epsilon decreases
        self.batch_size = batch_size
        self.memory = deque(maxlen=buffer_size) # Replay buffer
        self.target_update_freq = target_update_freq # How often to update target network (in training steps)
        # self.episode_counter = 0 # Use training steps for target update instead of episodes
        self.train_step_counter = 0 # Tracks number of optimizer steps

    def choose_action(self, state):
        """Chooses an action using an epsilon-greedy policy."""
        if random.random() < self.epsilon:
            return random.randint(0, self.output_dim - 1) # Explore
        else:
            # Exploit: Choose the best action based on the current Q-network
            if not isinstance(state, np.ndarray):
                state = np.array(state, dtype=np.float32)

            # Pre-check state for NaN/Inf before converting to tensor
            if np.isnan(state).any() or np.isinf(state).any():
                print("Warning: NaN/Inf detected in state array before choosing action. Replacing with 0.")
                state = np.nan_to_num(state, nan=0.0, posinf=0.0, neginf=0.0)

            try:
                 state_tensor = torch.FloatTensor(state).unsqueeze(0).to(self.device)
                 self.model.eval() # Set model to evaluation mode for inference
                 with torch.no_grad():
                      q_values = self.model(state_tensor)
                 self.model.train() # Set model back to training mode

                 # Check for NaN/Inf in Q-values
                 if torch.isnan(q_values).any() or torch.isinf(q_values).any():
                      print("Warning: NaN/Inf detected in Q-values output. Choosing random action.")
                      return random.randint(0, self.output_dim - 1)

                 action = int(torch.argmax(q_values).item())
                 return action

            except Exception as e:
                print(f"Error during action selection: {e}")
                print(f"State causing error: {state}")
                # Fallback to random action in case of error
                return random.randint(0, self.output_dim - 1)


    def store_transition(self, state, action, reward, next_state, done):
        """Stores a transition in the replay memory."""
        # Ensure all components are numpy arrays of the correct type before storing
        if not isinstance(state, np.ndarray): state = np.array(state, dtype=np.float32)
        if not isinstance(next_state, np.ndarray): next_state = np.array(next_state, dtype=np.float32)

        # Clean potential NaN/Inf values before storing
        state = np.nan_to_num(state, nan=0.0, posinf=0.0, neginf=0.0)
        next_state = np.nan_to_num(next_state, nan=0.0, posinf=0.0, neginf=0.0)
        reward = np.float32(np.nan_to_num(reward, nan=0.0, posinf=0.0, neginf=0.0)) # Clean reward too

        self.memory.append((state.astype(np.float32),
                           int(action), # Ensure action is int
                           reward,
                           next_state.astype(np.float32),
                           bool(done))) # Ensure done is bool

    def train_step(self):
        """Performs one step of training using a batch sampled from memory."""
        if len(self.memory) < self.batch_size * 10:
            return 0.0

        batch = random.sample(self.memory, self.batch_size)
        states, actions, rewards, next_states, dones = zip(*batch)

        try:
            # === FIX === NumPy 非依存に変更 #########################
            states      = torch.tensor(states,      dtype=torch.float32, device=self.device)
            next_states = torch.tensor(next_states, dtype=torch.float32, device=self.device)

            # =======================================================
            actions  = torch.LongTensor(actions).unsqueeze(1).to(self.device)
            rewards  = torch.FloatTensor(rewards).unsqueeze(1).to(self.device)
            dones    = torch.BoolTensor(dones).unsqueeze(1).to(self.device)

        except Exception as e:
            print(f"Error converting batch to tensors: {e}")
            return 0.0


        # --- Double DQN Logic ---
        self.model.eval() # Use main model to select best action for next state
        self.target_model.eval() # Use target model to evaluate the selected action

        with torch.no_grad():
             # Select the action for next_state using the online network (self.model)
             # Ensure next_states doesn't contain NaN/Inf before feeding to model
             if torch.isnan(next_states).any() or torch.isinf(next_states).any():
                  print("Warning: NaN/Inf detected in next_states tensor during training. Clamping.")
                  next_states = torch.nan_to_num(next_states, nan=0.0, posinf=1e6, neginf=-1e6)

             next_q_values_online = self.model(next_states)
             if torch.isnan(next_q_values_online).any() or torch.isinf(next_q_values_online).any():
                  print("Warning: NaN/Inf in next_q_values_online. Using zeros.")
                  best_next_actions = torch.zeros_like(actions) # Match shape and device
             else:
                  best_next_actions = next_q_values_online.argmax(dim=1, keepdim=True)


             # Evaluate the Q-value of the selected action using the target network (self.target_model)
             next_q_values_target = self.target_model(next_states)
             if torch.isnan(next_q_values_target).any() or torch.isinf(next_q_values_target).any():
                  print("Warning: NaN/Inf in next_q_values_target. Using zeros.")
                  next_q_value = torch.zeros_like(rewards) # Match shape and device
             else:
                  next_q_value = next_q_values_target.gather(1, best_next_actions)


             # Compute the target Q-value: R + gamma * Q_target(S', argmax_a Q_online(S', a))
             # Zero out Q-value for terminal states
             target_q = rewards + self.gamma * next_q_value * (~dones) # Use ~dones for masking

             # Check for NaN/Inf in target_q
             if torch.isnan(target_q).any() or torch.isinf(target_q).any():
                   print("Warning: NaN/Inf detected in target_q. Skipping loss calculation.")
                   target_q = torch.nan_to_num(target_q, nan=0.0, posinf=1e6, neginf=-1e6) # Clamp instead of skipping?
                   # return 0.0 # Or maybe clamp and proceed?

        # --- Calculate Loss ---
        self.model.train() # Set model back to training mode

        # Get current Q-values for the actions taken: Q_online(S, A)
        current_q = self.model(states).gather(1, actions)

        # Check for NaN/Inf in current_q
        if torch.isnan(current_q).any() or torch.isinf(current_q).any():
             print("Warning: NaN/Inf detected in current_q. Skipping loss calculation.")
             # Potentially inspect states/actions causing NaN
             return 0.0

        # Use Smooth L1 Loss (Huber Loss) - often more robust than MSE
        loss = nn.SmoothL1Loss()(current_q, target_q.detach()) # Detach target_q as it's considered fixed target

        # --- Optimization Step ---
        if torch.isnan(loss) or torch.isinf(loss):
            print("Warning: NaN/Inf detected in loss. Skipping optimizer step.")
            return 0.0 # Skip backpropagation if loss is invalid
        else:
            self.optimizer.zero_grad()
            loss.backward()
            # Gradient clipping to prevent exploding gradients
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
            self.optimizer.step()


        # --- Epsilon Decay and Target Network Update ---
        self.train_step_counter += 1

        # Decay epsilon based on training steps
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay # Exponential decay
            # Alternative: Linear decay
            # self.epsilon -= (initial_epsilon - self.epsilon_min) / total_decay_steps

        # Update target network periodically based on training steps
        if self.train_step_counter % self.target_update_freq == 0:
            print(f"\n--- Updating target network at training step {self.train_step_counter} ---")
            self.target_model.load_state_dict(self.model.state_dict())

        # --- LR Scheduling ---
        # Step the scheduler based on training steps (or episodes, depending on strategy)
        # Stepping per training step might decay LR too fast, consider stepping per episode end
        # self.scheduler.step() # Moved to end of episode in training loop

        return loss.item() # Return the loss value for logging


    def update_target_network_and_schedule_lr(self): # Renamed for clarity, called per episode end
        """Updates target network (if needed based on step counter) and steps LR scheduler."""
        # Target network update is handled within train_step based on self.train_step_counter
        # Step the learning rate scheduler at the end of each episode
        current_lr = self.optimizer.param_groups[0]['lr']
        self.scheduler.step()
        new_lr = self.optimizer.param_groups[0]['lr']
        # Optional: Log LR change
        # if new_lr != current_lr:
        #      print(f"LR stepped from {current_lr:.3E} to {new_lr:.3E}")


    def save_model(self, path):
        """Saves the model's state dictionary."""
        try:
            # Create directory if it doesn't exist
            os.makedirs(os.path.dirname(path), exist_ok=True)
            # Save model state
            torch.save(self.model.state_dict(), path)
            print(f"\nModel state saved to {path}")
        except Exception as e:
            print(f"\nError saving model state to {path}: {e}")

    def load_model(self, path):
        """Loads the model's state dictionary."""
        if os.path.exists(path):
            try:
                # Load state dict, ensuring it's loaded to the correct device
                state_dict = torch.load(path, map_location=self.device)
                self.model.load_state_dict(state_dict)
                # Also update the target network immediately after loading
                self.target_model.load_state_dict(self.model.state_dict())
                self.model.train() # Ensure model is in training mode after loading
                self.target_model.eval() # Ensure target model is in eval mode
                print(f"Model state loaded from {path}")
            except Exception as e:
                print(f"Error loading model state from {path}: {e}")
                print("Model weights were NOT loaded. Starting from scratch or previous state.")
        else:
            print(f"Model file not found at {path}. Starting from scratch.")

# --- Random Agent for Opponent ---
def random_agent_action(env):
    """Simple agent that chooses a random valid action."""
    # In this simple version, just return a random direction index 0-3
    return random.randint(0, 3)

# --- Time Formatting Function ---
def format_time(seconds):
    """Formats seconds into HH:MM:SS."""
    if seconds < 0: seconds = 0
    h = int(seconds // 3600)
    m = int((seconds % 3600) // 60)
    s = int(seconds % 60)
    return f"{h:02d}:{m:02d}:{s:02d}"

# === メイン学習ループ ===
if __name__ == '__main__':
    # --- Configuration ---
    NUM_EPISODES = 100 # Increase episodes for potentially more complex strategy
    BOARD_SIZE = 6
    MAX_TURNS = 40 # Max turns per game
    INPUT_DIM = BOARD_SIZE * BOARD_SIZE + 8 # 36 + 2*pos_self + 2*pos_opp + 1*moves + 1*top_self + 1*top_opp + 1*turn = 44
    OUTPUT_DIM = 4 # up, down, left, right

    # --- Agent Hyperparameters (May need tuning) ---
    LR = 1e-5 # Slightly lower learning rate
    GAMMA = 0.99
    EPSILON_START = 1.0
    EPSILON_MIN = 0.05 # Minimum exploration
    # Adjust decay steps based on NUM_EPISODES and desired exploration duration
    # Example: Reach EPSILON_MIN roughly halfway through training
    TOTAL_DECAY_FRAMES = NUM_EPISODES * MAX_TURNS * 0.75 # Approx frames to reach min epsilon
    EPSILON_DECAY_RATE = math.exp(math.log(EPSILON_MIN / EPSILON_START) / TOTAL_DECAY_FRAMES)
    # EPSILON_DECAY_RATE = 0.999998 # Or use a fixed decay rate per frame

    BUFFER_SIZE = 500_000 # Smaller buffer might focus on more recent experiences
    BATCH_SIZE = 128      # Smaller batch size
    TARGET_UPDATE_FREQ = 1000 # Update target network every N training steps
    LR_SCHEDULER_GAMMA = 0.9998 # Slower LR decay per episode

    TRAIN_START_FRAMES = BATCH_SIZE * 10 # Minimum steps in buffer before training starts
    TRAIN_FREQ = 1       # Train every N environment steps for the agent

    # --- Logging/Saving ---
    LOG_INTERVAL = 100   # Log progress every N episodes
    SAVE_INTERVAL = 5000 # Save model checkpoint every N episodes
    LOAD_EXISTING_MODEL = False # Set to True to load a pre-trained model
    MODEL_LOAD_PATH = os.path.join(save_dir, "dqn_model_final.pth") # Path to load from if LOAD_EXISTING_MODEL is True

    # --- Initialization ---
    env = PsychoDiceWarsEnv(board_size=BOARD_SIZE, max_turns=MAX_TURNS)
    agent = DQNAgent(input_dim=INPUT_DIM, output_dim=OUTPUT_DIM,
                     lr=LR, gamma=GAMMA, epsilon=EPSILON_START, epsilon_min=EPSILON_MIN,
                     epsilon_decay=EPSILON_DECAY_RATE, buffer_size=BUFFER_SIZE, batch_size=BATCH_SIZE,
                     target_update_freq=TARGET_UPDATE_FREQ, lr_scheduler_gamma=LR_SCHEDULER_GAMMA)

    if LOAD_EXISTING_MODEL:
        agent.load_model(MODEL_LOAD_PATH)
        # Optionally adjust epsilon if loading a trained model
        # agent.epsilon = agent.epsilon_min # Start with minimal exploration if loading a fully trained model

    log_path = os.path.join(save_dir, "training_log_strategic.txt")
    file_exists = os.path.exists(log_path)
    # Open in append mode ('a') if exists, write mode ('w') otherwise
    try:
        log_file = open(log_path, "a" if file_exists else "w", encoding="utf-8")
        if not file_exists or os.path.getsize(log_path) == 0:
            log_file.write(f"--- Training Start ({time.strftime('%Y-%m-%d %H:%M:%S')}) ---\n")
            log_file.write(f"Agent Params: LR={LR:.1E}, Gamma={GAMMA}, EpsDecay={EPSILON_DECAY_RATE:.6f} (rate per frame), EpsMin={EPSILON_MIN}, Buffer={BUFFER_SIZE//1000}k, Batch={BATCH_SIZE}, TargetUpdate(steps)={TARGET_UPDATE_FREQ}, LRSchedGamma(ep)={LR_SCHEDULER_GAMMA:.4f}\n")
            log_file.write(f"Env Params: Board={BOARD_SIZE}x{BOARD_SIZE}, MaxTurns={MAX_TURNS}\n")
            log_file.write(f"Training Params: Episodes={NUM_EPISODES//1000}k, TrainStartFrames={TRAIN_START_FRAMES}, TrainFreq={TRAIN_FREQ}\n")
            log_file.write(f"Load Model: {LOAD_EXISTING_MODEL}, Path: {MODEL_LOAD_PATH if LOAD_EXISTING_MODEL else 'N/A'}\n")
            log_file.write(f"Save Dir: {save_dir}\n")
            log_file.write("-" * 80 + "\n")
            log_file.flush() # Ensure header is written immediately
    except Exception as e:
        print(f"Error opening log file {log_path}: {e}")
        # Fallback to console output if log file fails
        log_file = None # Indicate logging is disabled


    start_time = time.time()
    total_env_steps = 0
    recent_rewards = deque(maxlen=LOG_INTERVAL)
    recent_losses = deque(maxlen=LOG_INTERVAL * MAX_TURNS) # Store more losses for smoother average
    recent_scores_w = deque(maxlen=LOG_INTERVAL)
    recent_scores_b = deque(maxlen=LOG_INTERVAL)
    recent_wins = deque(maxlen=LOG_INTERVAL)
    recent_kills = deque(maxlen=LOG_INTERVAL) # Track kills per episode

    print(f"Starting training for {NUM_EPISODES} episodes...")
    print(f"Logging detailed progress every {LOG_INTERVAL} episodes.")
    print(f"Saving model checkpoint every {SAVE_INTERVAL} episodes.")
    print(f"Reward structure includes: Move(0.05-0.3), Kill(+5), Top6End(+0.5), AdjacentEnd(-1), InvalidMove(-0.5), Win/Loss(+/-15 + diff*0.1)")

    # --- Training Loop ---
    pbar = tqdm(range(NUM_EPISODES), unit="ep", desc="Initializing...", ncols=120, leave=True)
    try: # Wrap main loop in try-except for cleaner exit on KeyboardInterrupt
        for ep in pbar:
            state = env.reset()
            # Initial state validation
            if np.isnan(state).any() or np.isinf(state).any():
                print(f"FATAL: Invalid initial state after reset at episode {ep+1}. Aborting.")
                if log_file: log_file.write(f"FATAL: Invalid initial state at episode {ep+1}. Aborting.\n")
                break

            done = False
            episode_reward_sum = 0.0
            episode_steps = 0
            episode_loss_sum = 0.0
            num_train_calls_ep = 0
            episode_kills = 0
            final_score_w, final_score_b = 0, 0
            is_win = False

            while not done:
                current_player_loop = env.current_player

                # --- Agent's Turn (Player 1) ---
                if current_player_loop == 1:
                    action = agent.choose_action(state)
                    next_state, reward, done, info = env.step(action)

                    total_env_steps += 1
                    episode_reward_sum += reward # Accumulate reward for the agent

                    # Store transition for the agent
                    agent.store_transition(state, action, reward, next_state, done)

                    # Training Step (if buffer is ready and training frequency allows)
                    if total_env_steps > TRAIN_START_FRAMES and total_env_steps % TRAIN_FREQ == 0:
                        loss = agent.train_step()
                        if loss > 0.0: # Only record actual training losses
                            episode_loss_sum += loss
                            num_train_calls_ep += 1
                            recent_losses.append(loss) # Add individual loss to recent losses

                    state = next_state # Update state for the next iteration (which might be Player 2's turn)

                    # Check if kill happened in this step (info from env.step)
                    if info.get('battle_result') == 'kill':
                         episode_kills += 1


                # --- Opponent's Turn (Player 2 - Random) ---
                else:
                    # Player 2 (random) takes its turn
                    # Note: We don't train Player 2 or store its transitions
                    while env.current_player == 2 and not done:
                        action_p2 = random_agent_action(env)
                        next_state, _, done, info_p2 = env.step(action_p2) # Reward for P2 is ignored
                        total_env_steps += 1 # Count opponent steps too

                        # Check if Player 2's turn ended or game ended
                        if info_p2.get('turn_ended', False) or done:
                             state = next_state # Update state for Player 1's next turn
                             break # Exit Player 2's move loop

                        # Safety break if P2 gets stuck somehow
                        if episode_steps > MAX_TURNS * BOARD_SIZE * 2: # Very generous step limit
                            print(f"Warning: Player 2 seems stuck in episode {ep+1}. Forcing turn end.")
                            if log_file: log_file.write(f"Warning: Player 2 stuck in ep {ep+1}\n")
                            # Manually switch turn or end game? For now, just break P2 loop.
                            env.current_player = 1 # Force switch back
                            state = env.get_state() # Get fresh state for P1
                            break

                episode_steps += 1 # Increment episode step counter

                # Check for game end condition within the loop
                if done:
                    if info.get('final_score') is not None:
                        final_score_w, final_score_b = info['final_score']
                        is_win = final_score_w > final_score_b
                    elif 'info_p2' in locals() and info_p2.get('final_score') is not None:
                        # If game ended on Player 2's turn
                        final_score_w, final_score_b = info_p2['final_score']
                        is_win = final_score_w > final_score_b
                    else:
                         # Game ended but no final score? Maybe error or early exit.
                         # Recalculate score just in case
                         score1 = np.sum(env.board == 1)
                         score2 = np.sum(env.board == 2)
                         final_score_w, final_score_b = score1, score2
                         is_win = final_score_w > final_score_b
                         print(f"Warning: Game ended without final_score in info at ep {ep+1}. Recalculated: W{final_score_w}/B{final_score_b}")


                # Safety break for excessively long episodes
                if episode_steps > MAX_TURNS * BOARD_SIZE * 2: # Adjusted threshold
                    print(f"Warning: Episode {ep+1} exceeded {MAX_TURNS * BOARD_SIZE * 2} steps. Force ending.")
                    if log_file: log_file.write(f"Warning: Episode {ep+1} force ended due to excessive steps.\n")
                    done = True # Force loop exit

            # --- End of Episode ---
            agent.update_target_network_and_schedule_lr() # Step LR scheduler once per episode

            # Store episode results
            recent_rewards.append(episode_reward_sum)
            recent_scores_w.append(final_score_w)
            recent_scores_b.append(final_score_b)
            recent_wins.append(1 if is_win else 0)
            recent_kills.append(episode_kills)
            # Note: recent_losses is appended during train_step

            # --- Logging ---
            if (ep + 1) % LOG_INTERVAL == 0:
                avg_reward = np.mean(recent_rewards) if recent_rewards else 0
                avg_loss = np.mean(recent_losses) if recent_losses else 0 # Avg loss over recent training steps
                avg_score_w = np.mean(recent_scores_w) if recent_scores_w else 0
                avg_score_b = np.mean(recent_scores_b) if recent_scores_b else 0
                win_rate = np.mean(recent_wins) * 100 if recent_wins else 0
                avg_kills = np.mean(recent_kills) if recent_kills else 0
                total_train_steps = agent.train_step_counter
                current_lr_log = agent.optimizer.param_groups[0]['lr']

                elapsed_time = time.time() - start_time
                progress_fraction = (ep + 1) / NUM_EPISODES
                if progress_fraction > 1e-6: # Avoid division by zero early on
                    estimated_total_time = elapsed_time / progress_fraction
                    estimated_remaining_time = estimated_total_time - elapsed_time
                else:
                    estimated_remaining_time = 0

                elapsed_time_str = format_time(elapsed_time)
                remaining_time_str = format_time(estimated_remaining_time)
                env_steps_per_sec = total_env_steps / elapsed_time if elapsed_time > 0 else 0
                train_steps_per_sec = total_train_steps / elapsed_time if elapsed_time > 0 else 0


                log_header = f"\n--- Progress Summary [Episode {ep+1}/{NUM_EPISODES}] ---"
                log_body = (
                    f"  Time -> Elapsed: {elapsed_time_str} | Remaining (Est.): {remaining_time_str} | Env Steps/s: {env_steps_per_sec:.1f} | Train Steps/s: {train_steps_per_sec:.1f}\n"
                    f"  Steps -> Env: {total_env_steps/1000:.1f}k | Train: {total_train_steps/1000:.1f}k\n"
                    f"  Performance (Avg over last {len(recent_rewards)} episodes):\n"
                    f"    Reward: {avg_reward:.3f} | Win Rate: {win_rate:.1f}% | Avg Kills: {avg_kills:.2f}\n"
                    f"    Score (W/B): {avg_score_w:.1f} / {avg_score_b:.1f} | Avg Loss: {avg_loss:.5f}\n"
                    f"  Agent State -> Epsilon: {agent.epsilon:.5f} | LR: {current_lr_log:.3E}\n"
                    f"  Memory Buffer -> Size: {len(agent.memory)}/{agent.memory.maxlen} ({(len(agent.memory)/agent.memory.maxlen*100):.1f}%)"
                )
                log_footer = "-" * 80

                print(log_header)
                print(log_body)
                print(log_footer)

                if log_file:
                    try:
                        log_file.write(log_header + "\n")
                        log_file.write(log_body + "\n")
                        log_file.write(log_footer + "\n")
                        log_file.flush() # Write logs to file periodically
                    except Exception as e:
                         print(f"Error writing to log file: {e}")


            # --- Model Saving ---
            if (ep + 1) % SAVE_INTERVAL == 0:
                interim_model_path = os.path.join(save_dir, f"dqn_model_ep{ep+1}.pth")
                agent.save_model(interim_model_path)
                if log_file:
                     try:
                        log_file.write(f"Checkpoint saved: {interim_model_path} at episode {ep+1}\n")
                        log_file.flush()
                     except Exception as e:
                         print(f"Error writing save notification to log file: {e}")

            # Update progress bar description
            desc_str = (f"AvgR:{np.mean(recent_rewards):.1f} "
                        f"Win:{np.mean(recent_wins)*100:.0f}% "
                        f"Loss:{np.mean(recent_losses):.4f} "
                        f"Eps:{agent.epsilon:.3f} "
                        f"LR:{agent.optimizer.param_groups[0]['lr']:.1E}")
            pbar.set_description(desc_str)

    except KeyboardInterrupt:
        print("\nTraining interrupted by user.")
        if log_file: log_file.write("\n--- Training Interrupted ---\n")

    finally:
        pbar.close() # Close the progress bar properly

        # --- Final Save and Summary ---
        final_model_path = os.path.join(save_dir, "dqn_model_final.pth")
        agent.save_model(final_model_path)

        total_training_time = time.time() - start_time
        final_msg = (
            f"\n=== Training { 'Completed' if ep + 1 == NUM_EPISODES else 'Stopped'} ===\n"
            f"Total Episodes Run: {ep + 1}\n"
            f"Total Env Steps: {total_env_steps}\n"
            f"Total Train Steps: {agent.train_step_counter}\n"
            f"Total Training Time: {format_time(total_training_time)}\n"
            f"Final Epsilon: {agent.epsilon:.5f}\n"
            f"Final Learning Rate: {agent.optimizer.param_groups[0]['lr']:.3E}\n"
            f"Final Model saved to: {final_model_path}\n"
            f"Log file saved to: {log_path if log_file else 'N/A'}\n"
            f"=========================="
        )
        print(final_msg)
        if log_file:
            try:
                log_file.write(final_msg + "\n")
                log_file.close() # Close the log file
            except Exception as e:
                 print(f"Error writing final message to log file or closing it: {e}")


    print("\nTraining script finished.")


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-package

Using device: cpu
Starting training for 100 episodes...
Logging detailed progress every 100 episodes.
Saving model checkpoint every 5000 episodes.
Reward structure includes: Move(0.05-0.3), Kill(+5), Top6End(+0.5), AdjacentEnd(-1), InvalidMove(-0.5), Win/Loss(+/-15 + diff*0.1)


Initializing...:   0%|                                                                          | 0/100 [00:00<?, ?ep/s]/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.11/dist-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
AvgR:9


--- Updating target network at training step 1000 ---


AvgR:11.7 Win:30% Loss:0.0703 Eps:0.138 LR:1.0E-05:  23%|██████▉                       | 23/100 [00:37<03:11,  2.49s/ep]

...


--- Updating target network at training step 11000 ---


AvgR:15.5 Win:15% Loss:0.0514 Eps:0.050 LR:9.8E-06:  94%|████████████████████████████▏ | 94/100 [03:46<00:16,  2.75s/ep]


--- Updating target network at training step 12000 ---


AvgR:15.5 Win:16% Loss:0.0513 Eps:0.050 LR:9.8E-06: 100%|█████████████████████████████| 100/100 [04:02<00:00,  2.43s/ep]


--- Progress Summary [Episode 100/100] ---
  Time -> Elapsed: 00:04:02 | Remaining (Est.): 00:00:00 | Env Steps/s: 115.6 | Train Steps/s: 52.5
  Steps -> Env: 28.0k | Train: 12.7k
  Performance (Avg over last 100 episodes):
    Reward: 15.514 | Win Rate: 16.0% | Avg Kills: 0.94
    Score (W/B): 12.4 / 20.1 | Avg Loss: 0.05129
  Agent State -> Epsilon: 0.04995 | LR: 9.802E-06
  Memory Buffer -> Size: 14016/500000 (2.8%)
--------------------------------------------------------------------------------

Model state saved to E:\OneDrive - 学校法人　永守学園\個人ファイル\Python to UnrealEngine5\DiceWars_EvenDeeper_Strategic/dqn_model_final.pth

=== Training Completed ===
Total Episodes Run: 100
Total Env Steps: 28025
Total Train Steps: 12737
Total Training Time: 00:04:02
Final Epsilon: 0.04995
Final Learning Rate: 9.802E-06
Final Model saved to: E:\OneDrive - 学校法人　永守学園\個人ファイル\Python to UnrealEngine5\DiceWars_EvenDeeper_Strategic/dqn_model_final.pth
Log file saved to: E:\OneDrive - 学校法人　永守学園\個人ファイル\Python 

学習済みデータVSランダム(player1 VS Player2)

In [3]:
import numpy as np
import random
import torch
import os
import time
from collections import deque

# ※　以下のクラス（Dice, PsychoDiceWarsEnv, DQN, DQNAgent, random_agent_action, format_time）
#     は、既に学習済みモデルのトレーニングコードに定義されているものを前提としています。
#     ここではシミュレーション用に再利用します。

# --- 盤面を６×６グリッド形式で表示する関数 ---
def print_board_grid(board):
    # board の各セルが 0:未塗装, 1:プレイヤー1, 2:プレイヤー2 として扱われる
    symbol = {0: "・", 1: "□", 2: "■"}
    print("盤面:")
    for r in range(board.shape[0]):
        row_str = ""
        for c in range(board.shape[1]):
            row_str += symbol.get(board[r, c], "?") + " "
        print(row_str)
    print("-" * 20)

# --- シミュレーション用関数 ---
def simulate_matches(num_matches=100, model_path="model.pth"):
    # 環境とエージェントの初期化（board_sizeやmax_turnsは適宜設定）
    env = PsychoDiceWarsEnv(board_size=6, max_turns=40)
    agent = DQNAgent(input_dim=44, output_dim=4)
    # 保存済みモデルのロード（学習済みパラメータを使用）
    agent.load_model(model_path)
    # シミュレーション時は探索を無効化
    agent.epsilon = 0.0

    # 全体の統計用変数
    match_results = []
    total_kills_agent = 0
    total_kills_random = 0
    total_control_agent = 0
    total_control_random = 0

    print("=== 100戦のシミュレーション開始 ===")
    for match in range(1, num_matches + 1):
        state = env.reset()
        done = False
        kill_count_agent = 0   # プレイヤー1（□）のキル数
        kill_count_random = 0  # プレイヤー2（■）のキル数

        # 試合中は各ターンごとに行動
        while not done:
            current_player = env.current_player
            if current_player == 1:
                action = agent.choose_action(state)
                next_state, reward, done, info = env.step(action)
                if info.get('battle_result') == 'kill':
                    kill_count_agent += 1
            else:
                action = random_agent_action(env)
                next_state, reward, done, info = env.step(action)
                if info.get('battle_result') == 'kill':
                    kill_count_random += 1
            state = next_state

        # 試合終了後：ボード上の色支配率（セル数）を計算
        board = env.board
        control_agent = np.sum(board == 1)
        control_random = np.sum(board == 2)
        total_cells = board.size
        control_rate_agent = control_agent / total_cells * 100
        control_rate_random = control_random / total_cells * 100

        # infoから最終スコアを取得し、勝者を判定
        final_score = info.get('final_score', (0, 0))
        if final_score[0] > final_score[1]:
            winner = "プレイヤー1 (□)"
        elif final_score[1] > final_score[0]:
            winner = "プレイヤー2 (■)"
        else:
            winner = "引き分け"

        # 各試合の統計を保存
        match_results.append({
            'match': match,
            'control_agent': control_rate_agent,
            'control_random': control_rate_random,
            'kill_agent': kill_count_agent,
            'kill_random': kill_count_random,
            'winner': winner
        })

        total_kills_agent += kill_count_agent
        total_kills_random += kill_count_random
        total_control_agent += control_rate_agent
        total_control_random += control_rate_random

        # 色支配の割合を□（プレイヤー1）と■（プレイヤー2）で簡易表示（全10マス換算）
        squares_total = 10
        squares_agent = int(round((control_rate_agent / 100) * squares_total))
        squares_random = squares_total - squares_agent

        # 各試合の結果をターミナルに出力
        print(f"試合 {match}: {'□' * squares_agent}{'■' * squares_random}  勝者: {winner}")
        print(f"  プレイヤー1 (□): 色支配率 {control_rate_agent:.1f}%、キル数: {kill_count_agent}")
        print(f"  プレイヤー2 (■): 色支配率 {control_rate_random:.1f}%、キル数: {kill_count_random}")
        # ここで盤面を6×6のマスで表示
        print_board_grid(board)

    # 全試合の平均統計を計算
    avg_control_agent = total_control_agent / num_matches
    avg_control_random = total_control_random / num_matches
    avg_kill_agent = total_kills_agent / num_matches
    avg_kill_random = total_kills_random / num_matches

    # シミュレーション全体の概要を出力
    print("=== 総合統計 ===")
    print(f"試合数: {num_matches}")
    print(f"プレイヤー1 (□): 平均色支配率 {avg_control_agent:.1f}%、平均キル数 {avg_kill_agent:.2f}")
    print(f"プレイヤー2 (■): 平均色支配率 {avg_control_random:.1f}%、平均キル数 {avg_kill_random:.2f}")
    print("※ 上記情報は、ゲームバランス調整やレベル設計の参考として利用できます。")

if __name__ == '__main__':
    # 学習済みモデルのパスを指定（例：最終モデルのパス）
    MODEL_LOAD_PATH = r"E:\OneDrive - 学校法人　永守学園\個人ファイル\Python to UnrealEngine5\DiceWars_EvenDeeper_Progress\dqn_model_final.pth"
    simulate_matches(num_matches=1000, model_path=MODEL_LOAD_PATH)


ストリーミング出力は最後の 5000 行に切り捨てられました。
--------------------
試合 547: □□■■■■■■■■  勝者: プレイヤー2 (■)
  プレイヤー1 (□): 色支配率 22.2%、キル数: 1
  プレイヤー2 (■): 色支配率 75.0%、キル数: 2
盤面:
■ ■ ■ ■ ■ ■ 
□ □ ■ ■ ■ ■ 
□ □ □ □ ■ ■ 
□ □ ■ ・ ■ ■ 
■ ■ ■ ■ ■ ■ 
■ ■ ■ ■ ■ ■ 
--------------------
試合 548: □■■■■■■■■■  勝者: プレイヤー2 (■)
  プレイヤー1 (□): 色支配率 5.6%、キル数: 0
  プレイヤー2 (■): 色支配率 83.3%、キル数: 2
盤面:
■ ■ ■ ■ ■ ■ 
■ ■ ■ □ ■ ■ 
■ ■ ■ ■ ■ ■ 
■ ■ ■ ■ ■ ・ 
□ ■ ■ ■ ■ ・ 
■ ■ ■ ■ ・ ・ 
--------------------
試合 549: □□■■■■■■■■  勝者: プレイヤー2 (■)
  プレイヤー1 (□): 色支配率 19.4%、キル数: 0
  プレイヤー2 (■): 色支配率 55.6%、キル数: 0
盤面:
□ □ ・ ■ ・ ・ 
□ □ □ □ □ ■ 
・ ・ ・ ■ ■ ■ 
■ ■ ・ ・ ■ ■ 
■ ■ ■ ■ ■ ■ 
■ ■ ・ ■ ■ ■ 
--------------------
試合 550: □□■■■■■■■■  勝者: プレイヤー2 (■)
  プレイヤー1 (□): 色支配率 16.7%、キル数: 0
  プレイヤー2 (■): 色支配率 66.7%、キル数: 0
盤面:
■ ■ ■ ・ ・ ・ 
・ ■ ■ ■ ■ ■ 
□ □ □ □ □ □ 
■ ■ ■ ■ ■ ■ 
■ ■ ■ ■ ■ ■ 
■ ■ ■ ■ ・ ・ 
--------------------
試合 551: □■■■■■■■■■  勝者: プレイヤー2 (■)
  プレイヤー1 (□): 色支配率 11.1%、キル数: 0
  プレイヤー2 (■): 色支配率 80.6%、キル数: 0
盤面:
■ ■ ■ ■ ■ ■ 
■ ■ ■ ・ ■ ■ 
■ ■ ■ ■ ■ ■

両方学習済みデータ　失敗キルされることを恐れて全く動かないことがある

In [ ]:
import numpy as np
import random
import torch
import os
import time
from collections import deque

# ※ 以下のクラス（Dice, PsychoDiceWarsEnv, DQN, DQNAgent, format_time）は
#    既存の学習済みコードに定義されているものとします。

# --- Dice.roll のパッチ適用：初期サイコロについて回転回数をカウント ---
if not hasattr(Dice, '__original_roll__'):
    Dice.__original_roll__ = Dice.roll

def patched_roll(self, direction):
    # is_initial が True の場合のみ回転回数をカウント
    if getattr(self, 'is_initial', False):
        if not hasattr(self, 'rotation_count'):
            self.rotation_count = 0
        self.rotation_count += 1
    return Dice.__original_roll__(self, direction)

Dice.roll = patched_roll

# --- 盤面を６×６グリッド形式で表示する関数 ---
def print_board_grid(board):
    # board の各セルが 0:未塗装, 1:プレイヤー1, 2:プレイヤー2 として扱われる
    symbol = {0: "・", 1: "□", 2: "■"}
    print("盤面:")
    for r in range(board.shape[0]):
        row_str = ""
        for c in range(board.shape[1]):
            row_str += symbol.get(board[r, c], "?") + " "
        print(row_str)
    print("-" * 20)

# --- シミュレーション用関数（両プレイヤーが学習済みモデルを使用） ---
def simulate_matches(num_matches=100, model_path="model.pth"):
    # 環境の初期化（board_size や max_turns は適宜設定）
    env = PsychoDiceWarsEnv(board_size=6, max_turns=40)

    # プレイヤー1用エージェント
    agent1 = DQNAgent(input_dim=44, output_dim=4)
    agent1.load_model(model_path)
    agent1.epsilon = 0.0  # 学習済みの方策に従う

    # プレイヤー2用エージェント
    agent2 = DQNAgent(input_dim=44, output_dim=4)
    agent2.load_model(model_path)
    agent2.epsilon = 0.0  # 学習済みの方策に従う

    # 全体の統計用変数
    match_results = []
    total_kills_agent1 = 0
    total_kills_agent2 = 0
    total_control_agent1 = 0
    total_control_agent2 = 0

    # 初期状態ごとの集計用辞書（キー：(初期上面, 初期位置)）
    # 各値は { 'count': 試合数, 'wins': 勝利数, 'total_control': 色支配率の総和, 'total_rotation': 回転回数の総和 }
    stats_agent1 = {}
    stats_agent2 = {}

    print("=== {}戦のシミュレーション開始（両プレイヤーが学習済みモデルを使用） ===".format(num_matches))
    for match in range(1, num_matches + 1):
        state = env.reset()
        # 試合開始直後に、初期サイコロの上面および初期位置を記録
        init_top_agent1 = env.dice1.top
        init_pos_agent1 = env.dice1.pos  # (row, col)
        init_top_agent2 = env.dice2.top
        init_pos_agent2 = env.dice2.pos

        # 初期サイコロとみなすためのフラグをセットし、回転回数を初期化
        env.dice1.is_initial = True
        env.dice2.is_initial = True
        env.dice1.rotation_count = 0
        env.dice2.rotation_count = 0

        done = False
        kill_count_agent1 = 0  # プレイヤー1（□）のキル数
        kill_count_agent2 = 0  # プレイヤー2（■）のキル数

        # 試合中は各ターンごとに行動
        while not done:
            current_player = env.current_player
            if current_player == 1:
                action = agent1.choose_action(state)
                next_state, reward, done, info = env.step(action)
                if info.get('battle_result') == 'kill':
                    kill_count_agent1 += 1
            else:
                action = agent2.choose_action(state)
                next_state, reward, done, info = env.step(action)
                if info.get('battle_result') == 'kill':
                    kill_count_agent2 += 1
            state = next_state

        # 試合終了時に、各プレイヤーの初期サイコロの回転回数を取得
        rotation_agent1 = getattr(env.dice1, 'rotation_count', 0)
        rotation_agent2 = getattr(env.dice2, 'rotation_count', 0)

        # 試合終了後：ボード上の色支配率（セル数）を計算
        board = env.board
        control_agent1 = np.sum(board == 1)
        control_agent2 = np.sum(board == 2)
        total_cells = board.size
        control_rate_agent1 = control_agent1 / total_cells * 100
        control_rate_agent2 = control_agent2 / total_cells * 100

        # info から最終スコアを取得し、勝者を判定
        final_score = info.get('final_score', (0, 0))
        if final_score[0] > final_score[1]:
            winner = "プレイヤー1 (□)"
            win_agent1 = 1
            win_agent2 = 0
        elif final_score[1] > final_score[0]:
            winner = "プレイヤー2 (■)"
            win_agent1 = 0
            win_agent2 = 1
        else:
            winner = "引き分け"
            win_agent1 = 0
            win_agent2 = 0

        # 各試合の統計を保存
        match_results.append({
            'match': match,
            'control_agent1': control_rate_agent1,
            'control_agent2': control_rate_agent2,
            'kill_agent1': kill_count_agent1,
            'kill_agent2': kill_count_agent2,
            'winner': winner,
            'init_top_agent1': init_top_agent1,
            'init_pos_agent1': init_pos_agent1,
            'rotation_agent1': rotation_agent1,
            'init_top_agent2': init_top_agent2,
            'init_pos_agent2': init_pos_agent2,
            'rotation_agent2': rotation_agent2
        })

        total_kills_agent1 += kill_count_agent1
        total_kills_agent2 += kill_count_agent2
        total_control_agent1 += control_rate_agent1
        total_control_agent2 += control_rate_agent2

        # 集計：プレイヤー1の場合
        key1 = (init_top_agent1, init_pos_agent1)
        if key1 not in stats_agent1:
            stats_agent1[key1] = {'count': 0, 'wins': 0, 'total_control': 0.0, 'total_rotation': 0}
        stats_agent1[key1]['count'] += 1
        stats_agent1[key1]['wins'] += win_agent1
        stats_agent1[key1]['total_control'] += control_rate_agent1
        stats_agent1[key1]['total_rotation'] += rotation_agent1

        # 集計：プレイヤー2の場合
        key2 = (init_top_agent2, init_pos_agent2)
        if key2 not in stats_agent2:
            stats_agent2[key2] = {'count': 0, 'wins': 0, 'total_control': 0.0, 'total_rotation': 0}
        stats_agent2[key2]['count'] += 1
        stats_agent2[key2]['wins'] += win_agent2
        stats_agent2[key2]['total_control'] += control_rate_agent2
        stats_agent2[key2]['total_rotation'] += rotation_agent2

        # 色支配の割合を□（プレイヤー1）と■（プレイヤー2）で簡易表示（全10マス換算）
        squares_total = 10
        squares_agent1_disp = int(round((control_rate_agent1 / 100) * squares_total))
        squares_agent2_disp = squares_total - squares_agent1_disp

        # 各試合の結果をターミナルに出力
        print(f"試合 {match}: {'□' * squares_agent1_disp}{'■' * squares_agent2_disp}  勝者: {winner}")
        print(f"  プレイヤー1 (□): 色支配率 {control_rate_agent1:.1f}%、キル数: {kill_count_agent1}")
        print(f"     初期上面: {init_top_agent1}、初期位置: {init_pos_agent1}、回転回数: {rotation_agent1}")
        print(f"  プレイヤー2 (■): 色支配率 {control_rate_agent2:.1f}%、キル数: {kill_count_agent2}")
        print(f"     初期上面: {init_top_agent2}、初期位置: {init_pos_agent2}、回転回数: {rotation_agent2}")
        print_board_grid(board)

    # 全試合の平均統計を計算
    avg_control_agent1 = total_control_agent1 / num_matches
    avg_control_agent2 = total_control_agent2 / num_matches
    avg_kill_agent1 = total_kills_agent1 / num_matches
    avg_kill_agent2 = total_kills_agent2 / num_matches

    # シミュレーション全体の概要を出力
    print("=== 総合統計 ===")
    print(f"試合数: {num_matches}")
    print(f"プレイヤー1 (□): 平均色支配率 {avg_control_agent1:.1f}%、平均キル数 {avg_kill_agent1:.2f}")
    print(f"プレイヤー2 (■): 平均色支配率 {avg_control_agent2:.1f}%、平均キル数 {avg_kill_agent2:.2f}")
    print("※ 上記情報は、ゲームバランス調整やレベル設計の参考として利用できます。")

    # 追加の集計結果を出力：初期サイコロの上面＆初期位置ごと
    print("\n=== プレイヤー1（□）初期状態別集計 ===")
    for key in sorted(stats_agent1.keys(), key=lambda x: (x[0], x[1])):
        count = stats_agent1[key]['count']
        wins = stats_agent1[key]['wins']
        avg_control = stats_agent1[key]['total_control'] / count
        avg_rotation = stats_agent1[key]['total_rotation'] / count
        win_rate = wins / count * 100
        print(f"初期上面 {key[0]}、初期位置 {key[1]}: 試合数 {count}, 勝率 {win_rate:.1f}%, 平均色支配率 {avg_control:.1f}%, 平均回転回数 {avg_rotation:.1f}")

    print("\n=== プレイヤー2（■）初期状態別集計 ===")
    for key in sorted(stats_agent2.keys(), key=lambda x: (x[0], x[1])):
        count = stats_agent2[key]['count']
        wins = stats_agent2[key]['wins']
        avg_control = stats_agent2[key]['total_control'] / count
        avg_rotation = stats_agent2[key]['total_rotation'] / count
        win_rate = wins / count * 100
        print(f"初期上面 {key[0]}、初期位置 {key[1]}: 試合数 {count}, 勝率 {win_rate:.1f}%, 平均色支配率 {avg_control:.1f}%, 平均回転回数 {avg_rotation:.1f}")

if __name__ == '__main__':
    # 学習済みモデルのパスを指定（例：最終モデルのパス）
    MODEL_LOAD_PATH = r"E:\OneDrive - 学校法人　永守学園\個人ファイル\Python to UnrealEngine5\DiceWars_EvenDeeper_Progress\dqn_model_final.pth"
    simulate_matches(num_matches=100000, model_path=MODEL_LOAD_PATH)



Streaming output truncated to the last 5000 lines.
盤面:
・ ・ ・ ・ ・ ・ 
・ ・ ・ ■ ■ ■ 
・ ・ ・ ・ ・ ・ 
・ ・ ・ ・ ・ ・ 
・ ・ ・ ・ ・ ・ 
□ □ ・ ・ ・ ・ 
--------------------
試合 30258: □■■■■■■■■■  勝者: 引き分け
  プレイヤー1 (□): 色支配率 8.3%、キル数: 0
     初期上面: 2、初期位置: (2, 2)、回転回数: 106
  プレイヤー2 (■): 色支配率 8.3%、キル数: 0
     初期上面: 1、初期位置: (4, 3)、回転回数: 105
盤面:
・ ・ ・ ・ ・ ・ 
・ ・ ・ ・ ・ ・ 
□ □ □ ・ ・ ・ 
・ ・ ・ ・ ・ ・ 
・ ・ ・ ■ ■ ■ 
・ ・ ・ ・ ・ ・ 
--------------------
試合 30259: ■■■■■■■■■■  勝者: プレイヤー2 (■)
  プレイヤー1 (□): 色支配率 2.8%、キル数: 0
     初期上面: 2、初期位置: (3, 0)、回転回数: 158
  プレイヤー2 (■): 色支配率 16.7%、キル数: 0
     初期上面: 3、初期位置: (0, 0)、回転回数: 158
盤面:
■ ■ ■ ■ ■ ■ 
・ ・ ・ ・ ・ ・ 
・ ・ ・ ・ ・ ・ 
□ ・ ・ ・ ・ ・ 
・ ・ ・ ・ ・ ・ 
・ ・ ・ ・ ・ ・ 
--------------------
試合 30260: □■■■■■■■■■  勝者: プレイヤー1 (□)
  プレイヤー1 (□): 色支配率 11.1%、キル数: 0
     初期上面: 5、初期位置: (1, 3)、回転回数: 161
  プレイヤー2 (■): 色支配率 5.6%、キル数: 0
     初期上面: 3、初期位置: (5, 4)、回転回数: 80
盤面:
・ ・ ・ ・ ・ ・ 
□ □ □ □ ・ ・ 
・ ・ ・ ・ ・ ・ 
・ ・ ・ ・ ・ ・ 
・ ・ ・ ・ ・ ・ 
・ ・ ・ ・ ■ ■ 
--------------------
試合 30261: □□■■■■■■■■  勝者: プレイ

=== 総合統計 ===
試合数: 100000
プレイヤー1 (□): 平均色支配率 9.6%、平均キル数 0.04
プレイヤー2 (■): 平均色支配率 9.7%、平均キル数 0.04
※ 上記情報は、ゲームバランス調整やレベル設計の参考として利用できます。

=== プレイヤー1（□）初期状態別集計 ===
初期上面 1、初期位置 (0, 0): 試合数 485, 勝率 0.0%, 平均色支配率 2.8%, 平均回転回数 127.4
初期上面 1、初期位置 (0, 1): 試合数 476, 勝率 19.5%, 平均色支配率 5.7%, 平均回転回数 126.3
初期上面 1、初期位置 (0, 2): 試合数 465, 勝率 38.5%, 平均色支配率 8.6%, 平均回転回数 124.6
初期上面 1、初期位置 (0, 3): 試合数 467, 勝率 53.5%, 平均色支配率 11.4%, 平均回転回数 123.4
初期上面 1、初期位置 (0, 4): 試合数 448, 勝率 72.8%, 平均色支配率 14.1%, 平均回転回数 124.6
初期上面 1、初期位置 (0, 5): 試合数 422, 勝率 100.0%, 平均色支配率 16.9%, 平均回転回数 126.1
初期上面 1、初期位置 (1, 0): 試合数 469, 勝率 0.0%, 平均色支配率 2.8%, 平均回転回数 124.3
初期上面 1、初期位置 (1, 1): 試合数 440, 勝率 20.2%, 平均色支配率 5.7%, 平均回転回数 121.8
初期上面 1、初期位置 (1, 2): 試合数 467, 勝率 39.6%, 平均色支配率 8.9%, 平均回転回数 125.2
初期上面 1、初期位置 (1, 3): 試合数 441, 勝率 54.4%, 平均色支配率 11.3%, 平均回転回数 122.6
初期上面 1、初期位置 (1, 4): 試合数 460, 勝率 80.0%, 平均色支配率 14.1%, 平均回転回数 124.1
初期上面 1、初期位置 (1, 5): 試合数 450, 勝率 93.6%, 平均色支配率 17.2%, 平均回転回数 122.0
初期上面 1、初期位置 (2, 0): 試合数 471, 勝率 0.0%, 平均色支配率 2.7%, 平均回転回数 121.5
初期上面 1、初期位置 (2, 1): 試合数 433, 勝率 22.9%, 平均色支配率 5.5%, 平均回転回数 124.1
初期上面 1、初期位置 (2, 2): 試合数 452, 勝率 39.6%, 平均色支配率 8.5%, 平均回転回数 124.9
初期上面 1、初期位置 (2, 3): 試合数 461, 勝率 58.6%, 平均色支配率 11.2%, 平均回転回数 124.8
初期上面 1、初期位置 (2, 4): 試合数 454, 勝率 77.3%, 平均色支配率 14.1%, 平均回転回数 124.0
初期上面 1、初期位置 (2, 5): 試合数 467, 勝率 87.4%, 平均色支配率 16.9%, 平均回転回数 122.1
初期上面 1、初期位置 (3, 0): 試合数 488, 勝率 0.6%, 平均色支配率 2.9%, 平均回転回数 119.4
初期上面 1、初期位置 (3, 1): 試合数 497, 勝率 19.7%, 平均色支配率 5.4%, 平均回転回数 123.5
初期上面 1、初期位置 (3, 2): 試合数 462, 勝率 41.8%, 平均色支配率 8.2%, 平均回転回数 122.4
初期上面 1、初期位置 (3, 3): 試合数 461, 勝率 56.6%, 平均色支配率 11.2%, 平均回転回数 124.9
初期上面 1、初期位置 (3, 4): 試合数 470, 勝率 66.2%, 平均色支配率 13.8%, 平均回転回数 124.4
初期上面 1、初期位置 (3, 5): 試合数 460, 勝率 84.6%, 平均色支配率 16.6%, 平均回転回数 122.8
初期上面 1、初期位置 (4, 0): 試合数 479, 勝率 0.4%, 平均色支配率 2.8%, 平均回転回数 121.2
初期上面 1、初期位置 (4, 1): 試合数 472, 勝率 17.2%, 平均色支配率 5.4%, 平均回転回数 122.1
初期上面 1、初期位置 (4, 2): 試合数 460, 勝率 34.6%, 平均色支配率 8.3%, 平均回転回数 122.8
初期上面 1、初期位置 (4, 3): 試合数 454, 勝率 44.5%, 平均色支配率 10.8%, 平均回転回数 120.9
初期上面 1、初期位置 (4, 4): 試合数 461, 勝率 66.4%, 平均色支配率 13.9%, 平均回転回数 124.0
初期上面 1、初期位置 (4, 5): 試合数 452, 勝率 80.5%, 平均色支配率 16.5%, 平均回転回数 120.1
初期上面 1、初期位置 (5, 0): 試合数 466, 勝率 2.1%, 平均色支配率 4.0%, 平均回転回数 101.4
初期上面 1、初期位置 (5, 1): 試合数 464, 勝率 14.7%, 平均色支配率 6.0%, 平均回転回数 104.3
初期上面 1、初期位置 (5, 2): 試合数 426, 勝率 25.4%, 平均色支配率 8.7%, 平均回転回数 103.5
初期上面 1、初期位置 (5, 3): 試合数 456, 勝率 42.1%, 平均色支配率 11.3%, 平均回転回数 102.2
初期上面 1、初期位置 (5, 4): 試合数 470, 勝率 52.1%, 平均色支配率 13.6%, 平均回転回数 100.3
初期上面 1、初期位置 (5, 5): 試合数 480, 勝率 63.3%, 平均色支配率 16.1%, 平均回転回数 102.0
初期上面 2、初期位置 (0, 0): 試合数 467, 勝率 0.0%, 平均色支配率 2.8%, 平均回転回数 144.9
初期上面 2、初期位置 (0, 1): 試合数 453, 勝率 13.7%, 平均色支配率 5.7%, 平均回転回数 144.7
初期上面 2、初期位置 (0, 2): 試合数 463, 勝率 36.7%, 平均色支配率 8.5%, 平均回転回数 144.6
初期上面 2、初期位置 (0, 3): 試合数 505, 勝率 52.1%, 平均色支配率 11.4%, 平均回転回数 146.2
初期上面 2、初期位置 (0, 4): 試合数 486, 勝率 72.6%, 平均色支配率 14.2%, 平均回転回数 145.9
初期上面 2、初期位置 (0, 5): 試合数 449, 勝率 100.0%, 平均色支配率 16.8%, 平均回転回数 148.8
初期上面 2、初期位置 (1, 0): 試合数 441, 勝率 0.2%, 平均色支配率 2.7%, 平均回転回数 144.7
初期上面 2、初期位置 (1, 1): 試合数 464, 勝率 21.1%, 平均色支配率 5.7%, 平均回転回数 146.7
初期上面 2、初期位置 (1, 2): 試合数 491, 勝率 33.4%, 平均色支配率 8.3%, 平均回転回数 146.3
初期上面 2、初期位置 (1, 3): 試合数 472, 勝率 52.8%, 平均色支配率 11.2%, 平均回転回数 144.6
初期上面 2、初期位置 (1, 4): 試合数 458, 勝率 81.9%, 平均色支配率 14.1%, 平均回転回数 144.4
初期上面 2、初期位置 (1, 5): 試合数 438, 勝率 91.8%, 平均色支配率 17.0%, 平均回転回数 145.2
初期上面 2、初期位置 (2, 0): 試合数 447, 勝率 0.0%, 平均色支配率 2.7%, 平均回転回数 146.1
初期上面 2、初期位置 (2, 1): 試合数 467, 勝率 18.2%, 平均色支配率 5.5%, 平均回転回数 142.6
初期上面 2、初期位置 (2, 2): 試合数 492, 勝率 37.2%, 平均色支配率 8.3%, 平均回転回数 141.9
初期上面 2、初期位置 (2, 3): 試合数 482, 勝率 58.1%, 平均色支配率 11.2%, 平均回転回数 143.9
初期上面 2、初期位置 (2, 4): 試合数 451, 勝率 71.0%, 平均色支配率 13.7%, 平均回転回数 146.5
初期上面 2、初期位置 (2, 5): 試合数 463, 勝率 77.8%, 平均色支配率 16.5%, 平均回転回数 144.1
初期上面 2、初期位置 (3, 0): 試合数 460, 勝率 0.0%, 平均色支配率 2.6%, 平均回転回数 145.2
初期上面 2、初期位置 (3, 1): 試合数 470, 勝率 20.4%, 平均色支配率 5.5%, 平均回転回数 144.5
初期上面 2、初期位置 (3, 2): 試合数 464, 勝率 37.5%, 平均色支配率 8.0%, 平均回転回数 139.2
初期上面 2、初期位置 (3, 3): 試合数 468, 勝率 53.2%, 平均色支配率 11.0%, 平均回転回数 143.0
初期上面 2、初期位置 (3, 4): 試合数 460, 勝率 63.9%, 平均色支配率 13.8%, 平均回転回数 143.5
初期上面 2、初期位置 (3, 5): 試合数 471, 勝率 78.1%, 平均色支配率 16.2%, 平均回転回数 142.2
初期上面 2、初期位置 (4, 0): 試合数 463, 勝率 0.0%, 平均色支配率 2.7%, 平均回転回数 140.0
初期上面 2、初期位置 (4, 1): 試合数 452, 勝率 18.4%, 平均色支配率 5.2%, 平均回転回数 140.8
初期上面 2、初期位置 (4, 2): 試合数 511, 勝率 29.7%, 平均色支配率 7.9%, 平均回転回数 141.5
初期上面 2、初期位置 (4, 3): 試合数 474, 勝率 40.9%, 平均色支配率 10.6%, 平均回転回数 141.5
初期上面 2、初期位置 (4, 4): 試合数 437, 勝率 58.6%, 平均色支配率 13.1%, 平均回転回数 139.7
初期上面 2、初期位置 (4, 5): 試合数 441, 勝率 74.8%, 平均色支配率 16.0%, 平均回転回数 141.9
初期上面 2、初期位置 (5, 0): 試合数 440, 勝率 4.5%, 平均色支配率 4.1%, 平均回転回数 123.9
初期上面 2、初期位置 (5, 1): 試合数 453, 勝率 15.5%, 平均色支配率 6.0%, 平均回転回数 127.1
初期上面 2、初期位置 (5, 2): 試合数 452, 勝率 22.1%, 平均色支配率 8.1%, 平均回転回数 130.7
初期上面 2、初期位置 (5, 3): 試合数 492, 勝率 32.5%, 平均色支配率 10.8%, 平均回転回数 122.3
初期上面 2、初期位置 (5, 4): 試合数 481, 勝率 52.0%, 平均色支配率 13.5%, 平均回転回数 121.2
初期上面 2、初期位置 (5, 5): 試合数 482, 勝率 68.9%, 平均色支配率 15.9%, 平均回転回数 122.1
初期上面 3、初期位置 (0, 0): 試合数 445, 勝率 0.0%, 平均色支配率 2.8%, 平均回転回数 139.7
初期上面 3、初期位置 (0, 1): 試合数 505, 勝率 18.0%, 平均色支配率 5.7%, 平均回転回数 137.5
初期上面 3、初期位置 (0, 2): 試合数 507, 勝率 35.1%, 平均色支配率 8.5%, 平均回転回数 138.9
初期上面 3、初期位置 (0, 3): 試合数 461, 勝率 53.1%, 平均色支配率 11.3%, 平均回転回数 136.3
初期上面 3、初期位置 (0, 4): 試合数 460, 勝率 70.0%, 平均色支配率 14.0%, 平均回転回数 138.6
初期上面 3、初期位置 (0, 5): 試合数 432, 勝率 100.0%, 平均色支配率 16.9%, 平均回転回数 140.9
初期上面 3、初期位置 (1, 0): 試合数 424, 勝率 0.0%, 平均色支配率 2.7%, 平均回転回数 137.2
初期上面 3、初期位置 (1, 1): 試合数 452, 勝率 19.9%, 平均色支配率 5.5%, 平均回転回数 137.8
初期上面 3、初期位置 (1, 2): 試合数 450, 勝率 36.9%, 平均色支配率 8.6%, 平均回転回数 137.4
初期上面 3、初期位置 (1, 3): 試合数 451, 勝率 55.7%, 平均色支配率 11.4%, 平均回転回数 138.0
初期上面 3、初期位置 (1, 4): 試合数 477, 勝率 80.5%, 平均色支配率 13.9%, 平均回転回数 138.8
初期上面 3、初期位置 (1, 5): 試合数 502, 勝率 90.8%, 平均色支配率 17.0%, 平均回転回数 139.2
初期上面 3、初期位置 (2, 0): 試合数 444, 勝率 0.2%, 平均色支配率 2.7%, 平均回転回数 136.3
初期上面 3、初期位置 (2, 1): 試合数 419, 勝率 19.3%, 平均色支配率 5.7%, 平均回転回数 137.6
初期上面 3、初期位置 (2, 2): 試合数 454, 勝率 36.1%, 平均色支配率 8.2%, 平均回転回数 141.2
初期上面 3、初期位置 (2, 3): 試合数 435, 勝率 59.8%, 平均色支配率 11.0%, 平均回転回数 138.0
初期上面 3、初期位置 (2, 4): 試合数 487, 勝率 70.4%, 平均色支配率 13.6%, 平均回転回数 138.5
初期上面 3、初期位置 (2, 5): 試合数 406, 勝率 71.9%, 平均色支配率 16.3%, 平均回転回数 137.9
初期上面 3、初期位置 (3, 0): 試合数 465, 勝率 0.2%, 平均色支配率 2.7%, 平均回転回数 137.7
初期上面 3、初期位置 (3, 1): 試合数 512, 勝率 15.8%, 平均色支配率 5.2%, 平均回転回数 135.9
初期上面 3、初期位置 (3, 2): 試合数 464, 勝率 37.7%, 平均色支配率 8.1%, 平均回転回数 137.5
初期上面 3、初期位置 (3, 3): 試合数 512, 勝率 48.0%, 平均色支配率 10.6%, 平均回転回数 135.7
初期上面 3、初期位置 (3, 4): 試合数 473, 勝率 52.9%, 平均色支配率 13.2%, 平均回転回数 136.1
初期上面 3、初期位置 (3, 5): 試合数 457, 勝率 72.2%, 平均色支配率 16.3%, 平均回転回数 135.3
初期上面 3、初期位置 (4, 0): 試合数 493, 勝率 0.4%, 平均色支配率 2.7%, 平均回転回数 135.4
初期上面 3、初期位置 (4, 1): 試合数 492, 勝率 17.1%, 平均色支配率 5.4%, 平均回転回数 134.6
初期上面 3、初期位置 (4, 2): 試合数 472, 勝率 30.3%, 平均色支配率 7.7%, 平均回転回数 135.2
初期上面 3、初期位置 (4, 3): 試合数 468, 勝率 37.8%, 平均色支配率 10.3%, 平均回転回数 133.2
初期上面 3、初期位置 (4, 4): 試合数 491, 勝率 55.6%, 平均色支配率 13.0%, 平均回転回数 133.6
初期上面 3、初期位置 (4, 5): 試合数 488, 勝率 71.9%, 平均色支配率 15.7%, 平均回転回数 135.1
初期上面 3、初期位置 (5, 0): 試合数 463, 勝率 3.5%, 平均色支配率 3.9%, 平均回転回数 122.9
初期上面 3、初期位置 (5, 1): 試合数 511, 勝率 13.9%, 平均色支配率 5.9%, 平均回転回数 119.4
初期上面 3、初期位置 (5, 2): 試合数 441, 勝率 27.0%, 平均色支配率 8.4%, 平均回転回数 117.5
初期上面 3、初期位置 (5, 3): 試合数 425, 勝率 40.0%, 平均色支配率 10.6%, 平均回転回数 111.6
初期上面 3、初期位置 (5, 4): 試合数 499, 勝率 50.3%, 平均色支配率 13.6%, 平均回転回数 116.3
初期上面 3、初期位置 (5, 5): 試合数 430, 勝率 64.4%, 平均色支配率 15.9%, 平均回転回数 115.6
初期上面 4、初期位置 (0, 0): 試合数 456, 勝率 0.0%, 平均色支配率 2.8%, 平均回転回数 160.0
初期上面 4、初期位置 (0, 1): 試合数 474, 勝率 21.7%, 平均色支配率 5.7%, 平均回転回数 160.0
初期上面 4、初期位置 (0, 2): 試合数 487, 勝率 39.0%, 平均色支配率 8.4%, 平均回転回数 160.0
初期上面 4、初期位置 (0, 3): 試合数 433, 勝率 55.0%, 平均色支配率 11.2%, 平均回転回数 160.0
初期上面 4、初期位置 (0, 4): 試合数 439, 勝率 69.7%, 平均色支配率 14.0%, 平均回転回数 160.0
初期上面 4、初期位置 (0, 5): 試合数 469, 勝率 100.0%, 平均色支配率 16.8%, 平均回転回数 160.0
初期上面 4、初期位置 (1, 0): 試合数 430, 勝率 0.0%, 平均色支配率 2.7%, 平均回転回数 159.3
初期上面 4、初期位置 (1, 1): 試合数 437, 勝率 19.0%, 平均色支配率 5.9%, 平均回転回数 158.9
初期上面 4、初期位置 (1, 2): 試合数 435, 勝率 38.6%, 平均色支配率 8.4%, 平均回転回数 160.0
初期上面 4、初期位置 (1, 3): 試合数 448, 勝率 55.1%, 平均色支配率 11.0%, 平均回転回数 159.3
初期上面 4、初期位置 (1, 4): 試合数 458, 勝率 81.9%, 平均色支配率 13.7%, 平均回転回数 160.0
初期上面 4、初期位置 (1, 5): 試合数 479, 勝率 85.8%, 平均色支配率 16.7%, 平均回転回数 160.0
初期上面 4、初期位置 (2, 0): 試合数 458, 勝率 0.0%, 平均色支配率 2.7%, 平均回転回数 158.6
初期上面 4、初期位置 (2, 1): 試合数 468, 勝率 16.0%, 平均色支配率 5.7%, 平均回転回数 158.6
初期上面 4、初期位置 (2, 2): 試合数 471, 勝率 38.4%, 平均色支配率 8.3%, 平均回転回数 158.0
初期上面 4、初期位置 (2, 3): 試合数 468, 勝率 60.9%, 平均色支配率 10.9%, 平均回転回数 157.9
初期上面 4、初期位置 (2, 4): 試合数 459, 勝率 61.7%, 平均色支配率 13.4%, 平均回転回数 157.6
初期上面 4、初期位置 (2, 5): 試合数 494, 勝率 70.6%, 平均色支配率 16.4%, 平均回転回数 158.7
初期上面 4、初期位置 (3, 0): 試合数 482, 勝率 0.0%, 平均色支配率 2.7%, 平均回転回数 158.0
初期上面 4、初期位置 (3, 1): 試合数 517, 勝率 21.5%, 平均色支配率 5.5%, 平均回転回数 157.5
初期上面 4、初期位置 (3, 2): 試合数 519, 勝率 41.4%, 平均色支配率 8.1%, 平均回転回数 156.0
初期上面 4、初期位置 (3, 3): 試合数 415, 勝率 48.2%, 平均色支配率 10.6%, 平均回転回数 157.7
初期上面 4、初期位置 (3, 4): 試合数 500, 勝率 49.6%, 平均色支配率 12.9%, 平均回転回数 158.4
初期上面 4、初期位置 (3, 5): 試合数 479, 勝率 68.5%, 平均色支配率 16.0%, 平均回転回数 157.0
初期上面 4、初期位置 (4, 0): 試合数 448, 勝率 0.2%, 平均色支配率 2.7%, 平均回転回数 156.1
初期上面 4、初期位置 (4, 1): 試合数 451, 勝率 18.6%, 平均色支配率 5.3%, 平均回転回数 155.7
初期上面 4、初期位置 (4, 2): 試合数 492, 勝率 32.3%, 平均色支配率 7.7%, 平均回転回数 158.0
初期上面 4、初期位置 (4, 3): 試合数 449, 勝率 37.4%, 平均色支配率 10.1%, 平均回転回数 156.1
初期上面 4、初期位置 (4, 4): 試合数 462, 勝率 51.1%, 平均色支配率 12.8%, 平均回転回数 154.8
初期上面 4、初期位置 (4, 5): 試合数 421, 勝率 70.1%, 平均色支配率 15.9%, 平均回転回数 155.8
初期上面 4、初期位置 (5, 0): 試合数 455, 勝率 2.9%, 平均色支配率 4.0%, 平均回転回数 135.0
初期上面 4、初期位置 (5, 1): 試合数 497, 勝率 13.3%, 平均色支配率 6.0%, 平均回転回数 137.5
初期上面 4、初期位置 (5, 2): 試合数 475, 勝率 26.3%, 平均色支配率 8.4%, 平均回転回数 132.4
初期上面 4、初期位置 (5, 3): 試合数 469, 勝率 37.7%, 平均色支配率 10.7%, 平均回転回数 134.8
初期上面 4、初期位置 (5, 4): 試合数 481, 勝率 54.5%, 平均色支配率 13.2%, 平均回転回数 135.1
初期上面 4、初期位置 (5, 5): 試合数 463, 勝率 61.1%, 平均色支配率 15.6%, 平均回転回数 133.4
初期上面 5、初期位置 (0, 0): 試合数 479, 勝率 0.0%, 平均色支配率 2.8%, 平均回転回数 149.0
初期上面 5、初期位置 (0, 1): 試合数 464, 勝率 18.3%, 平均色支配率 5.7%, 平均回転回数 147.1
初期上面 5、初期位置 (0, 2): 試合数 444, 勝率 39.9%, 平均色支配率 8.5%, 平均回転回数 146.6
初期上面 5、初期位置 (0, 3): 試合数 438, 勝率 52.7%, 平均色支配率 11.3%, 平均回転回数 149.4
初期上面 5、初期位置 (0, 4): 試合数 482, 勝率 75.3%, 平均色支配率 14.1%, 平均回転回数 146.8
初期上面 5、初期位置 (0, 5): 試合数 468, 勝率 100.0%, 平均色支配率 16.7%, 平均回転回数 148.0
初期上面 5、初期位置 (1, 0): 試合数 427, 勝率 0.0%, 平均色支配率 2.7%, 平均回転回数 147.4
初期上面 5、初期位置 (1, 1): 試合数 482, 勝率 18.3%, 平均色支配率 5.5%, 平均回転回数 147.9
初期上面 5、初期位置 (1, 2): 試合数 444, 勝率 36.3%, 平均色支配率 8.4%, 平均回転回数 146.3
初期上面 5、初期位置 (1, 3): 試合数 462, 勝率 54.3%, 平均色支配率 11.2%, 平均回転回数 146.2
初期上面 5、初期位置 (1, 4): 試合数 438, 勝率 83.1%, 平均色支配率 13.7%, 平均回転回数 149.2
初期上面 5、初期位置 (1, 5): 試合数 403, 勝率 83.4%, 平均色支配率 16.4%, 平均回転回数 144.2
初期上面 5、初期位置 (2, 0): 試合数 419, 勝率 0.2%, 平均色支配率 2.7%, 平均回転回数 149.2
初期上面 5、初期位置 (2, 1): 試合数 479, 勝率 20.0%, 平均色支配率 5.6%, 平均回転回数 146.6
初期上面 5、初期位置 (2, 2): 試合数 479, 勝率 39.5%, 平均色支配率 8.3%, 平均回転回数 146.0
初期上面 5、初期位置 (2, 3): 試合数 446, 勝率 62.1%, 平均色支配率 10.8%, 平均回転回数 146.2
初期上面 5、初期位置 (2, 4): 試合数 395, 勝率 68.9%, 平均色支配率 13.6%, 平均回転回数 143.6
初期上面 5、初期位置 (2, 5): 試合数 463, 勝率 67.8%, 平均色支配率 16.2%, 平均回転回数 144.6
初期上面 5、初期位置 (3, 0): 試合数 432, 勝率 0.0%, 平均色支配率 2.7%, 平均回転回数 144.3
初期上面 5、初期位置 (3, 1): 試合数 504, 勝率 19.2%, 平均色支配率 5.4%, 平均回転回数 144.0
初期上面 5、初期位置 (3, 2): 試合数 435, 勝率 41.4%, 平均色支配率 8.3%, 平均回転回数 145.0
初期上面 5、初期位置 (3, 3): 試合数 465, 勝率 46.0%, 平均色支配率 10.3%, 平均回転回数 144.7
初期上面 5、初期位置 (3, 4): 試合数 436, 勝率 46.1%, 平均色支配率 13.0%, 平均回転回数 144.7
初期上面 5、初期位置 (3, 5): 試合数 438, 勝率 66.0%, 平均色支配率 15.4%, 平均回転回数 144.3
初期上面 5、初期位置 (4, 0): 試合数 452, 勝率 0.2%, 平均色支配率 2.6%, 平均回転回数 148.0
初期上面 5、初期位置 (4, 1): 試合数 472, 勝率 19.9%, 平均色支配率 5.5%, 平均回転回数 145.4
初期上面 5、初期位置 (4, 2): 試合数 473, 勝率 30.9%, 平均色支配率 7.8%, 平均回転回数 143.8
初期上面 5、初期位置 (4, 3): 試合数 434, 勝率 34.6%, 平均色支配率 10.1%, 平均回転回数 143.8
初期上面 5、初期位置 (4, 4): 試合数 461, 勝率 50.8%, 平均色支配率 12.7%, 平均回転回数 144.6
初期上面 5、初期位置 (4, 5): 試合数 485, 勝率 67.4%, 平均色支配率 15.4%, 平均回転回数 143.4
初期上面 5、初期位置 (5, 0): 試合数 454, 勝率 2.2%, 平均色支配率 3.8%, 平均回転回数 124.7
初期上面 5、初期位置 (5, 1): 試合数 496, 勝率 15.9%, 平均色支配率 6.1%, 平均回転回数 125.8
初期上面 5、初期位置 (5, 2): 試合数 471, 勝率 27.2%, 平均色支配率 8.5%, 平均回転回数 127.3
初期上面 5、初期位置 (5, 3): 試合数 439, 勝率 40.1%, 平均色支配率 10.4%, 平均回転回数 126.5
初期上面 5、初期位置 (5, 4): 試合数 457, 勝率 56.7%, 平均色支配率 13.1%, 平均回転回数 126.3
初期上面 5、初期位置 (5, 5): 試合数 463, 勝率 67.8%, 平均色支配率 15.2%, 平均回転回数 125.0
初期上面 6、初期位置 (0, 0): 試合数 463, 勝率 0.0%, 平均色支配率 2.8%, 平均回転回数 128.7
初期上面 6、初期位置 (0, 1): 試合数 457, 勝率 22.1%, 平均色支配率 5.6%, 平均回転回数 131.5
初期上面 6、初期位置 (0, 2): 試合数 476, 勝率 37.2%, 平均色支配率 8.5%, 平均回転回数 126.3
初期上面 6、初期位置 (0, 3): 試合数 475, 勝率 49.9%, 平均色支配率 11.3%, 平均回転回数 131.3
初期上面 6、初期位置 (0, 4): 試合数 463, 勝率 70.6%, 平均色支配率 14.0%, 平均回転回数 129.7
初期上面 6、初期位置 (0, 5): 試合数 466, 勝率 100.0%, 平均色支配率 16.8%, 平均回転回数 125.1
初期上面 6、初期位置 (1, 0): 試合数 495, 勝率 0.4%, 平均色支配率 2.8%, 平均回転回数 128.0
初期上面 6、初期位置 (1, 1): 試合数 516, 勝率 16.9%, 平均色支配率 5.7%, 平均回転回数 126.8
初期上面 6、初期位置 (1, 2): 試合数 458, 勝率 39.1%, 平均色支配率 8.5%, 平均回転回数 130.7
初期上面 6、初期位置 (1, 3): 試合数 500, 勝率 55.0%, 平均色支配率 11.0%, 平均回転回数 128.0
初期上面 6、初期位置 (1, 4): 試合数 442, 勝率 82.8%, 平均色支配率 14.0%, 平均回転回数 126.5
初期上面 6、初期位置 (1, 5): 試合数 440, 勝率 83.4%, 平均色支配率 16.5%, 平均回転回数 127.8
初期上面 6、初期位置 (2, 0): 試合数 462, 勝率 0.2%, 平均色支配率 2.7%, 平均回転回数 129.5
初期上面 6、初期位置 (2, 1): 試合数 427, 勝率 18.0%, 平均色支配率 5.8%, 平均回転回数 126.0
初期上面 6、初期位置 (2, 2): 試合数 455, 勝率 37.8%, 平均色支配率 8.1%, 平均回転回数 127.6
初期上面 6、初期位置 (2, 3): 試合数 450, 勝率 58.4%, 平均色支配率 10.9%, 平均回転回数 127.2
初期上面 6、初期位置 (2, 4): 試合数 486, 勝率 66.7%, 平均色支配率 13.5%, 平均回転回数 126.8
初期上面 6、初期位置 (2, 5): 試合数 471, 勝率 61.6%, 平均色支配率 15.9%, 平均回転回数 125.9
初期上面 6、初期位置 (3, 0): 試合数 441, 勝率 0.0%, 平均色支配率 2.7%, 平均回転回数 127.1
初期上面 6、初期位置 (3, 1): 試合数 432, 勝率 16.9%, 平均色支配率 5.4%, 平均回転回数 125.2
初期上面 6、初期位置 (3, 2): 試合数 446, 勝率 39.0%, 平均色支配率 8.1%, 平均回転回数 124.0
初期上面 6、初期位置 (3, 3): 試合数 501, 勝率 52.1%, 平均色支配率 10.6%, 平均回転回数 127.6
初期上面 6、初期位置 (3, 4): 試合数 501, 勝率 48.5%, 平均色支配率 13.0%, 平均回転回数 123.8
初期上面 6、初期位置 (3, 5): 試合数 475, 勝率 67.4%, 平均色支配率 15.6%, 平均回転回数 125.6
初期上面 6、初期位置 (4, 0): 試合数 494, 勝率 0.0%, 平均色支配率 2.7%, 平均回転回数 127.1
初期上面 6、初期位置 (4, 1): 試合数 476, 勝率 22.3%, 平均色支配率 5.4%, 平均回転回数 129.7
初期上面 6、初期位置 (4, 2): 試合数 451, 勝率 32.8%, 平均色支配率 7.9%, 平均回転回数 122.0
初期上面 6、初期位置 (4, 3): 試合数 476, 勝率 38.0%, 平均色支配率 10.3%, 平均回転回数 124.3
初期上面 6、初期位置 (4, 4): 試合数 476, 勝率 55.5%, 平均色支配率 12.7%, 平均回転回数 126.8
初期上面 6、初期位置 (4, 5): 試合数 470, 勝率 65.1%, 平均色支配率 15.2%, 平均回転回数 126.8
初期上面 6、初期位置 (5, 0): 試合数 432, 勝率 3.2%, 平均色支配率 3.9%, 平均回転回数 109.6
初期上面 6、初期位置 (5, 1): 試合数 452, 勝率 15.5%, 平均色支配率 5.8%, 平均回転回数 109.7
初期上面 6、初期位置 (5, 2): 試合数 443, 勝率 28.4%, 平均色支配率 8.4%, 平均回転回数 106.8
初期上面 6、初期位置 (5, 3): 試合数 460, 勝率 39.1%, 平均色支配率 10.6%, 平均回転回数 107.7
初期上面 6、初期位置 (5, 4): 試合数 459, 勝率 54.9%, 平均色支配率 13.0%, 平均回転回数 111.1
初期上面 6、初期位置 (5, 5): 試合数 463, 勝率 70.4%, 平均色支配率 15.7%, 平均回転回数 105.5

=== プレイヤー2（■）初期状態別集計 ===
初期上面 1、初期位置 (0, 0): 試合数 422, 勝率 73.7%, 平均色支配率 17.2%, 平均回転回数 100.4
初期上面 1、初期位置 (0, 1): 試合数 473, 勝率 87.3%, 平均色支配率 16.2%, 平均回転回数 122.7
初期上面 1、初期位置 (0, 2): 試合数 456, 勝率 95.8%, 平均色支配率 16.6%, 平均回転回数 122.7
初期上面 1、初期位置 (0, 3): 試合数 477, 勝率 95.0%, 平均色支配率 16.6%, 平均回転回数 121.7
初期上面 1、初期位置 (0, 4): 試合数 513, 勝率 98.2%, 平均色支配率 16.7%, 平均回転回数 122.1
初期上面 1、初期位置 (0, 5): 試合数 436, 勝率 100.0%, 平均色支配率 16.7%, 平均回転回数 126.0
初期上面 1、初期位置 (1, 0): 試合数 454, 勝率 54.6%, 平均色支配率 14.4%, 平均回転回数 99.3
初期上面 1、初期位置 (1, 1): 試合数 453, 勝率 75.7%, 平均色支配率 13.8%, 平均回転回数 122.5
初期上面 1、初期位置 (1, 2): 試合数 441, 勝率 71.7%, 平均色支配率 13.9%, 平均回転回数 123.3
初期上面 1、初期位置 (1, 3): 試合数 477, 勝率 77.1%, 平均色支配率 13.7%, 平均回転回数 123.0
初期上面 1、初期位置 (1, 4): 試合数 453, 勝率 80.4%, 平均色支配率 13.8%, 平均回転回数 121.3
初期上面 1、初期位置 (1, 5): 試合数 472, 勝率 67.2%, 平均色支配率 13.9%, 平均回転回数 124.5
初期上面 1、初期位置 (2, 0): 試合数 480, 勝率 42.7%, 平均色支配率 12.3%, 平均回転回数 97.4
初期上面 1、初期位置 (2, 1): 試合数 437, 勝率 53.5%, 平均色支配率 11.0%, 平均回転回数 121.3
初期上面 1、初期位置 (2, 2): 試合数 491, 勝率 61.5%, 平均色支配率 11.2%, 平均回転回数 124.1
初期上面 1、初期位置 (2, 3): 試合数 480, 勝率 62.7%, 平均色支配率 11.2%, 平均回転回数 125.4
初期上面 1、初期位置 (2, 4): 試合数 433, 勝率 55.9%, 平均色支配率 11.0%, 平均回転回数 124.3
初期上面 1、初期位置 (2, 5): 試合数 462, 勝率 48.9%, 平均色支配率 11.1%, 平均回転回数 123.7
初期上面 1、初期位置 (3, 0): 試合数 493, 勝率 29.6%, 平均色支配率 10.0%, 平均回転回数 102.2
初期上面 1、初期位置 (3, 1): 試合数 531, 勝率 38.2%, 平均色支配率 8.3%, 平均回転回数 122.1
初期上面 1、初期位置 (3, 2): 試合数 465, 勝率 40.0%, 平均色支配率 8.5%, 平均回転回数 124.4
初期上面 1、初期位置 (3, 3): 試合数 483, 勝率 31.9%, 平均色支配率 8.2%, 平均回転回数 124.6
初期上面 1、初期位置 (3, 4): 試合数 472, 勝率 33.9%, 平均色支配率 8.5%, 平均回転回数 124.8
初期上面 1、初期位置 (3, 5): 試合数 461, 勝率 36.2%, 平均色支配率 8.3%, 平均回転回数 123.4
初期上面 1、初期位置 (4, 0): 試合数 452, 勝率 16.6%, 平均色支配率 6.6%, 平均回転回数 103.5
初期上面 1、初期位置 (4, 1): 試合数 451, 勝率 20.2%, 平均色支配率 5.4%, 平均回転回数 119.9
初期上面 1、初期位置 (4, 2): 試合数 452, 勝率 17.5%, 平均色支配率 5.5%, 平均回転回数 126.4
初期上面 1、初期位置 (4, 3): 試合数 472, 勝率 16.5%, 平均色支配率 5.7%, 平均回転回数 121.7
初期上面 1、初期位置 (4, 4): 試合数 446, 勝率 19.1%, 平均色支配率 5.6%, 平均回転回数 123.8
初期上面 1、初期位置 (4, 5): 試合数 475, 勝率 16.8%, 平均色支配率 5.6%, 平均回転回数 126.7
初期上面 1、初期位置 (5, 0): 試合数 470, 勝率 2.8%, 平均色支配率 3.9%, 平均回転回数 108.9
初期上面 1、初期位置 (5, 1): 試合数 411, 勝率 0.2%, 平均色支配率 2.8%, 平均回転回数 119.9
初期上面 1、初期位置 (5, 2): 試合数 482, 勝率 0.0%, 平均色支配率 2.7%, 平均回転回数 122.2
初期上面 1、初期位置 (5, 3): 試合数 477, 勝率 0.0%, 平均色支配率 2.8%, 平均回転回数 124.7
初期上面 1、初期位置 (5, 4): 試合数 412, 勝率 0.0%, 平均色支配率 2.7%, 平均回転回数 122.4
初期上面 1、初期位置 (5, 5): 試合数 454, 勝率 0.0%, 平均色支配率 2.8%, 平均回転回数 122.3
初期上面 2、初期位置 (0, 0): 試合数 487, 勝率 73.1%, 平均色支配率 16.7%, 平均回転回数 119.5
初期上面 2、初期位置 (0, 1): 試合数 462, 勝率 89.2%, 平均色支配率 16.3%, 平均回転回数 142.3
初期上面 2、初期位置 (0, 2): 試合数 425, 勝率 91.3%, 平均色支配率 16.5%, 平均回転回数 141.9
初期上面 2、初期位置 (0, 3): 試合数 433, 勝率 94.0%, 平均色支配率 16.5%, 平均回転回数 143.6
初期上面 2、初期位置 (0, 4): 試合数 474, 勝率 95.8%, 平均色支配率 16.6%, 平均回転回数 145.7
初期上面 2、初期位置 (0, 5): 試合数 480, 勝率 100.0%, 平均色支配率 16.7%, 平均回転回数 146.1
初期上面 2、初期位置 (1, 0): 試合数 474, 勝率 58.6%, 平均色支配率 14.8%, 平均回転回数 117.7
初期上面 2、初期位置 (1, 1): 試合数 427, 勝率 71.9%, 平均色支配率 13.6%, 平均回転回数 140.6
初期上面 2、初期位置 (1, 2): 試合数 453, 勝率 73.3%, 平均色支配率 13.7%, 平均回転回数 141.5
初期上面 2、初期位置 (1, 3): 試合数 432, 勝率 79.2%, 平均色支配率 13.8%, 平均回転回数 144.7
初期上面 2、初期位置 (1, 4): 試合数 498, 勝率 78.3%, 平均色支配率 14.0%, 平均回転回数 144.4
初期上面 2、初期位置 (1, 5): 試合数 437, 勝率 68.6%, 平均色支配率 13.9%, 平均回転回数 146.3
初期上面 2、初期位置 (2, 0): 試合数 492, 勝率 42.1%, 平均色支配率 11.9%, 平均回転回数 119.2
初期上面 2、初期位置 (2, 1): 試合数 467, 勝率 56.3%, 平均色支配率 11.0%, 平均回転回数 139.5
初期上面 2、初期位置 (2, 2): 試合数 461, 勝率 57.7%, 平均色支配率 11.1%, 平均回転回数 145.3
初期上面 2、初期位置 (2, 3): 試合数 508, 勝率 66.1%, 平均色支配率 11.1%, 平均回転回数 145.3
初期上面 2、初期位置 (2, 4): 試合数 428, 勝率 49.5%, 平均色支配率 11.0%, 平均回転回数 146.5
初期上面 2、初期位置 (2, 5): 試合数 480, 勝率 48.3%, 平均色支配率 11.1%, 平均回転回数 145.6
初期上面 2、初期位置 (3, 0): 試合数 459, 勝率 32.0%, 平均色支配率 8.9%, 平均回転回数 123.2
初期上面 2、初期位置 (3, 1): 試合数 481, 勝率 41.4%, 平均色支配率 8.1%, 平均回転回数 143.0
初期上面 2、初期位置 (3, 2): 試合数 464, 勝率 37.9%, 平均色支配率 8.3%, 平均回転回数 141.3
初期上面 2、初期位置 (3, 3): 試合数 490, 勝率 35.9%, 平均色支配率 8.3%, 平均回転回数 143.1
初期上面 2、初期位置 (3, 4): 試合数 511, 勝率 36.8%, 平均色支配率 8.3%, 平均回転回数 145.6
初期上面 2、初期位置 (3, 5): 試合数 481, 勝率 35.6%, 平均色支配率 8.3%, 平均回転回数 146.0
初期上面 2、初期位置 (4, 0): 試合数 477, 勝率 17.8%, 平均色支配率 6.6%, 平均回転回数 125.5
初期上面 2、初期位置 (4, 1): 試合数 415, 勝率 21.2%, 平均色支配率 5.4%, 平均回転回数 141.4
初期上面 2、初期位置 (4, 2): 試合数 457, 勝率 17.5%, 平均色支配率 5.5%, 平均回転回数 143.3
初期上面 2、初期位置 (4, 3): 試合数 455, 勝率 17.1%, 平均色支配率 5.5%, 平均回転回数 145.0
初期上面 2、初期位置 (4, 4): 試合数 469, 勝率 15.6%, 平均色支配率 5.5%, 平均回転回数 144.6
初期上面 2、初期位置 (4, 5): 試合数 446, 勝率 17.3%, 平均色支配率 5.6%, 平均回転回数 146.3
初期上面 2、初期位置 (5, 0): 試合数 494, 勝率 3.4%, 平均色支配率 3.8%, 平均回転回数 127.0
初期上面 2、初期位置 (5, 1): 試合数 483, 勝率 0.4%, 平均色支配率 2.7%, 平均回転回数 142.0
初期上面 2、初期位置 (5, 2): 試合数 465, 勝率 0.6%, 平均色支配率 2.8%, 平均回転回数 142.6
初期上面 2、初期位置 (5, 3): 試合数 439, 勝率 0.0%, 平均色支配率 2.7%, 平均回転回数 144.6
初期上面 2、初期位置 (5, 4): 試合数 494, 勝率 0.0%, 平均色支配率 2.8%, 平均回転回数 144.3
初期上面 2、初期位置 (5, 5): 試合数 458, 勝率 0.0%, 平均色支配率 2.8%, 平均回転回数 145.2
初期上面 3、初期位置 (0, 0): 試合数 453, 勝率 71.1%, 平均色支配率 16.8%, 平均回転回数 115.9
初期上面 3、初期位置 (0, 1): 試合数 480, 勝率 87.3%, 平均色支配率 16.4%, 平均回転回数 134.9
初期上面 3、初期位置 (0, 2): 試合数 462, 勝率 88.5%, 平均色支配率 16.5%, 平均回転回数 137.8
初期上面 3、初期位置 (0, 3): 試合数 474, 勝率 93.2%, 平均色支配率 16.6%, 平均回転回数 135.3
初期上面 3、初期位置 (0, 4): 試合数 458, 勝率 97.6%, 平均色支配率 16.7%, 平均回転回数 141.1
初期上面 3、初期位置 (0, 5): 試合数 470, 勝率 100.0%, 平均色支配率 16.7%, 平均回転回数 138.3
初期上面 3、初期位置 (1, 0): 試合数 472, 勝率 56.6%, 平均色支配率 14.4%, 平均回転回数 116.3
初期上面 3、初期位置 (1, 1): 試合数 441, 勝率 71.7%, 平均色支配率 13.5%, 平均回転回数 139.0
初期上面 3、初期位置 (1, 2): 試合数 443, 勝率 77.2%, 平均色支配率 13.8%, 平均回転回数 136.9
初期上面 3、初期位置 (1, 3): 試合数 464, 勝率 80.4%, 平均色支配率 13.9%, 平均回転回数 139.8
初期上面 3、初期位置 (1, 4): 試合数 472, 勝率 82.2%, 平均色支配率 13.9%, 平均回転回数 140.0
初期上面 3、初期位置 (1, 5): 試合数 445, 勝率 71.5%, 平均色支配率 13.9%, 平均回転回数 138.6
初期上面 3、初期位置 (2, 0): 試合数 437, 勝率 46.0%, 平均色支配率 11.3%, 平均回転回数 118.6
初期上面 3、初期位置 (2, 1): 試合数 447, 勝率 52.3%, 平均色支配率 10.6%, 平均回転回数 140.9
初期上面 3、初期位置 (2, 2): 試合数 448, 勝率 62.1%, 平均色支配率 11.0%, 平均回転回数 139.5
初期上面 3、初期位置 (2, 3): 試合数 454, 勝率 61.9%, 平均色支配率 11.0%, 平均回転回数 137.1
初期上面 3、初期位置 (2, 4): 試合数 453, 勝率 52.8%, 平均色支配率 11.1%, 平均回転回数 137.3
初期上面 3、初期位置 (2, 5): 試合数 457, 勝率 49.9%, 平均色支配率 11.1%, 平均回転回数 138.0
初期上面 3、初期位置 (3, 0): 試合数 485, 勝率 34.4%, 平均色支配率 8.9%, 平均回転回数 119.3
初期上面 3、初期位置 (3, 1): 試合数 445, 勝率 33.9%, 平均色支配率 8.0%, 平均回転回数 137.3
初期上面 3、初期位置 (3, 2): 試合数 458, 勝率 42.6%, 平均色支配率 8.3%, 平均回転回数 135.3
初期上面 3、初期位置 (3, 3): 試合数 467, 勝率 37.3%, 平均色支配率 8.3%, 平均回転回数 137.0
初期上面 3、初期位置 (3, 4): 試合数 470, 勝率 35.5%, 平均色支配率 8.3%, 平均回転回数 134.4
初期上面 3、初期位置 (3, 5): 試合数 513, 勝率 31.6%, 平均色支配率 8.3%, 平均回転回数 138.7
初期上面 3、初期位置 (4, 0): 試合数 464, 勝率 15.1%, 平均色支配率 6.7%, 平均回転回数 117.7
初期上面 3、初期位置 (4, 1): 試合数 466, 勝率 19.5%, 平均色支配率 5.5%, 平均回転回数 136.8
初期上面 3、初期位置 (4, 2): 試合数 460, 勝率 15.9%, 平均色支配率 5.5%, 平均回転回数 138.3
初期上面 3、初期位置 (4, 3): 試合数 455, 勝率 15.8%, 平均色支配率 5.5%, 平均回転回数 137.0
初期上面 3、初期位置 (4, 4): 試合数 481, 勝率 17.0%, 平均色支配率 5.5%, 平均回転回数 139.0
初期上面 3、初期位置 (4, 5): 試合数 446, 勝率 16.4%, 平均色支配率 5.6%, 平均回転回数 137.8
初期上面 3、初期位置 (5, 0): 試合数 445, 勝率 2.9%, 平均色支配率 3.9%, 平均回転回数 116.0
初期上面 3、初期位置 (5, 1): 試合数 475, 勝率 0.2%, 平均色支配率 2.7%, 平均回転回数 138.0
初期上面 3、初期位置 (5, 2): 試合数 463, 勝率 0.0%, 平均色支配率 2.8%, 平均回転回数 134.5
初期上面 3、初期位置 (5, 3): 試合数 431, 勝率 0.0%, 平均色支配率 2.7%, 平均回転回数 135.8
初期上面 3、初期位置 (5, 4): 試合数 458, 勝率 0.0%, 平均色支配率 2.7%, 平均回転回数 139.1
初期上面 3、初期位置 (5, 5): 試合数 451, 勝率 0.0%, 平均色支配率 2.8%, 平均回転回数 139.4
初期上面 4、初期位置 (0, 0): 試合数 486, 勝率 72.6%, 平均色支配率 16.9%, 平均回転回数 135.3
初期上面 4、初期位置 (0, 1): 試合数 475, 勝率 83.2%, 平均色支配率 16.3%, 平均回転回数 155.6
初期上面 4、初期位置 (0, 2): 試合数 449, 勝率 88.6%, 平均色支配率 16.5%, 平均回転回数 158.6
初期上面 4、初期位置 (0, 3): 試合数 462, 勝率 91.1%, 平均色支配率 16.4%, 平均回転回数 157.9
初期上面 4、初期位置 (0, 4): 試合数 451, 勝率 97.3%, 平均色支配率 16.7%, 平均回転回数 160.0
初期上面 4、初期位置 (0, 5): 試合数 428, 勝率 100.0%, 平均色支配率 16.7%, 平均回転回数 160.0
初期上面 4、初期位置 (1, 0): 試合数 424, 勝率 55.4%, 平均色支配率 13.6%, 平均回転回数 137.0
初期上面 4、初期位置 (1, 1): 試合数 470, 勝率 65.5%, 平均色支配率 13.4%, 平均回転回数 155.6
初期上面 4、初期位置 (1, 2): 試合数 474, 勝率 71.1%, 平均色支配率 13.5%, 平均回転回数 157.0
初期上面 4、初期位置 (1, 3): 試合数 479, 勝率 74.9%, 平均色支配率 13.7%, 平均回転回数 158.3
初期上面 4、初期位置 (1, 4): 試合数 476, 勝率 83.2%, 平均色支配率 13.9%, 平均回転回数 158.7
初期上面 4、初期位置 (1, 5): 試合数 479, 勝率 68.5%, 平均色支配率 13.9%, 平均回転回数 160.0
初期上面 4、初期位置 (2, 0): 試合数 467, 勝率 41.5%, 平均色支配率 11.1%, 平均回転回数 134.3
初期上面 4、初期位置 (2, 1): 試合数 454, 勝率 55.1%, 平均色支配率 10.9%, 平均回転回数 154.7
初期上面 4、初期位置 (2, 2): 試合数 442, 勝率 57.5%, 平均色支配率 11.0%, 平均回転回数 158.6
初期上面 4、初期位置 (2, 3): 試合数 458, 勝率 59.0%, 平均色支配率 11.0%, 平均回転回数 157.9
初期上面 4、初期位置 (2, 4): 試合数 486, 勝率 51.4%, 平均色支配率 11.0%, 平均回転回数 159.7
初期上面 4、初期位置 (2, 5): 試合数 434, 勝率 50.5%, 平均色支配率 11.1%, 平均回転回数 160.0
初期上面 4、初期位置 (3, 0): 試合数 477, 勝率 34.0%, 平均色支配率 8.7%, 平均回転回数 134.8
初期上面 4、初期位置 (3, 1): 試合数 471, 勝率 35.9%, 平均色支配率 8.1%, 平均回転回数 156.9
初期上面 4、初期位置 (3, 2): 試合数 470, 勝率 38.7%, 平均色支配率 8.2%, 平均回転回数 156.3
初期上面 4、初期位置 (3, 3): 試合数 481, 勝率 37.0%, 平均色支配率 8.4%, 平均回転回数 159.3
初期上面 4、初期位置 (3, 4): 試合数 445, 勝率 30.1%, 平均色支配率 8.3%, 平均回転回数 158.2
初期上面 4、初期位置 (3, 5): 試合数 431, 勝率 32.9%, 平均色支配率 8.3%, 平均回転回数 160.0
初期上面 4、初期位置 (4, 0): 試合数 482, 勝率 16.4%, 平均色支配率 6.8%, 平均回転回数 134.4
初期上面 4、初期位置 (4, 1): 試合数 442, 勝率 22.9%, 平均色支配率 5.5%, 平均回転回数 154.6
初期上面 4、初期位置 (4, 2): 試合数 467, 勝率 18.6%, 平均色支配率 5.5%, 平均回転回数 158.6
初期上面 4、初期位置 (4, 3): 試合数 451, 勝率 20.2%, 平均色支配率 5.6%, 平均回転回数 158.2
初期上面 4、初期位置 (4, 4): 試合数 425, 勝率 19.3%, 平均色支配率 5.5%, 平均回転回数 159.2
初期上面 4、初期位置 (4, 5): 試合数 445, 勝率 13.0%, 平均色支配率 5.6%, 平均回転回数 160.0
初期上面 4、初期位置 (5, 0): 試合数 465, 勝率 1.9%, 平均色支配率 3.6%, 平均回転回数 142.5
初期上面 4、初期位置 (5, 1): 試合数 491, 勝率 0.4%, 平均色支配率 2.7%, 平均回転回数 157.4
初期上面 4、初期位置 (5, 2): 試合数 529, 勝率 0.2%, 平均色支配率 2.8%, 平均回転回数 157.0
初期上面 4、初期位置 (5, 3): 試合数 449, 勝率 0.0%, 平均色支配率 2.7%, 平均回転回数 159.3
初期上面 4、初期位置 (5, 4): 試合数 457, 勝率 0.0%, 平均色支配率 2.7%, 平均回転回数 158.2
初期上面 4、初期位置 (5, 5): 試合数 467, 勝率 0.0%, 平均色支配率 2.8%, 平均回転回数 160.0
初期上面 5、初期位置 (0, 0): 試合数 430, 勝率 67.4%, 平均色支配率 15.8%, 平均回転回数 121.8
初期上面 5、初期位置 (0, 1): 試合数 487, 勝率 84.2%, 平均色支配率 16.1%, 平均回転回数 145.7
初期上面 5、初期位置 (0, 2): 試合数 455, 勝率 87.3%, 平均色支配率 16.3%, 平均回転回数 143.1
初期上面 5、初期位置 (0, 3): 試合数 488, 勝率 91.4%, 平均色支配率 16.5%, 平均回転回数 147.5
初期上面 5、初期位置 (0, 4): 試合数 440, 勝率 97.3%, 平均色支配率 16.6%, 平均回転回数 149.2
初期上面 5、初期位置 (0, 5): 試合数 476, 勝率 100.0%, 平均色支配率 16.7%, 平均回転回数 147.5
初期上面 5、初期位置 (1, 0): 試合数 458, 勝率 59.4%, 平均色支配率 13.6%, 平均回転回数 120.5
初期上面 5、初期位置 (1, 1): 試合数 470, 勝率 70.6%, 平均色支配率 13.5%, 平均回転回数 142.7
初期上面 5、初期位置 (1, 2): 試合数 481, 勝率 76.5%, 平均色支配率 13.6%, 平均回転回数 148.6
初期上面 5、初期位置 (1, 3): 試合数 534, 勝率 77.3%, 平均色支配率 13.7%, 平均回転回数 144.9
初期上面 5、初期位置 (1, 4): 試合数 461, 勝率 75.7%, 平均色支配率 13.9%, 平均回転回数 145.6
初期上面 5、初期位置 (1, 5): 試合数 425, 勝率 67.3%, 平均色支配率 13.9%, 平均回転回数 147.3
初期上面 5、初期位置 (2, 0): 試合数 472, 勝率 45.1%, 平均色支配率 11.1%, 平均回転回数 122.2
初期上面 5、初期位置 (2, 1): 試合数 451, 勝率 53.9%, 平均色支配率 10.6%, 平均回転回数 142.2
初期上面 5、初期位置 (2, 2): 試合数 458, 勝率 57.4%, 平均色支配率 10.9%, 平均回転回数 145.2
初期上面 5、初期位置 (2, 3): 試合数 463, 勝率 59.6%, 平均色支配率 11.1%, 平均回転回数 147.0
初期上面 5、初期位置 (2, 4): 試合数 470, 勝率 49.4%, 平均色支配率 11.1%, 平均回転回数 148.2
初期上面 5、初期位置 (2, 5): 試合数 448, 勝率 56.2%, 平均色支配率 11.1%, 平均回転回数 149.7
初期上面 5、初期位置 (3, 0): 試合数 463, 勝率 34.1%, 平均色支配率 9.1%, 平均回転回数 125.1
初期上面 5、初期位置 (3, 1): 試合数 478, 勝率 41.8%, 平均色支配率 8.3%, 平均回転回数 144.0
初期上面 5、初期位置 (3, 2): 試合数 470, 勝率 44.5%, 平均色支配率 8.3%, 平均回転回数 145.2
初期上面 5、初期位置 (3, 3): 試合数 462, 勝率 34.4%, 平均色支配率 8.3%, 平均回転回数 146.2
初期上面 5、初期位置 (3, 4): 試合数 429, 勝率 36.8%, 平均色支配率 8.3%, 平均回転回数 147.6
初期上面 5、初期位置 (3, 5): 試合数 452, 勝率 40.5%, 平均色支配率 8.3%, 平均回転回数 147.7
初期上面 5、初期位置 (4, 0): 試合数 464, 勝率 15.7%, 平均色支配率 6.4%, 平均回転回数 120.8
初期上面 5、初期位置 (4, 1): 試合数 455, 勝率 20.2%, 平均色支配率 5.7%, 平均回転回数 143.6
初期上面 5、初期位置 (4, 2): 試合数 495, 勝率 17.4%, 平均色支配率 5.5%, 平均回転回数 146.2
初期上面 5、初期位置 (4, 3): 試合数 488, 勝率 15.8%, 平均色支配率 5.5%, 平均回転回数 146.6
初期上面 5、初期位置 (4, 4): 試合数 436, 勝率 16.5%, 平均色支配率 5.6%, 平均回転回数 145.2
初期上面 5、初期位置 (4, 5): 試合数 487, 勝率 16.0%, 平均色支配率 5.6%, 平均回転回数 147.7
初期上面 5、初期位置 (5, 0): 試合数 464, 勝率 2.8%, 平均色支配率 3.9%, 平均回転回数 131.0
初期上面 5、初期位置 (5, 1): 試合数 479, 勝率 0.4%, 平均色支配率 2.7%, 平均回転回数 146.3
初期上面 5、初期位置 (5, 2): 試合数 436, 勝率 0.2%, 平均色支配率 2.7%, 平均回転回数 142.7
初期上面 5、初期位置 (5, 3): 試合数 439, 勝率 0.0%, 平均色支配率 2.8%, 平均回転回数 145.3
初期上面 5、初期位置 (5, 4): 試合数 438, 勝率 0.0%, 平均色支配率 2.7%, 平均回転回数 145.3
初期上面 5、初期位置 (5, 5): 試合数 466, 勝率 0.0%, 平均色支配率 2.8%, 平均回転回数 147.8
初期上面 6、初期位置 (0, 0): 試合数 472, 勝率 69.5%, 平均色支配率 15.8%, 平均回転回数 105.5
初期上面 6、初期位置 (0, 1): 試合数 458, 勝率 81.7%, 平均色支配率 16.2%, 平均回転回数 126.5
初期上面 6、初期位置 (0, 2): 試合数 477, 勝率 82.2%, 平均色支配率 16.1%, 平均回転回数 129.0
初期上面 6、初期位置 (0, 3): 試合数 460, 勝率 91.1%, 平均色支配率 16.5%, 平均回転回数 129.3
初期上面 6、初期位置 (0, 4): 試合数 453, 勝率 97.4%, 平均色支配率 16.6%, 平均回転回数 128.8
初期上面 6、初期位置 (0, 5): 試合数 505, 勝率 100.0%, 平均色支配率 16.7%, 平均回転回数 126.8
初期上面 6、初期位置 (1, 0): 試合数 476, 勝率 58.4%, 平均色支配率 13.6%, 平均回転回数 105.7
初期上面 6、初期位置 (1, 1): 試合数 427, 勝率 65.6%, 平均色支配率 13.2%, 平均回転回数 124.0
初期上面 6、初期位置 (1, 2): 試合数 464, 勝率 75.9%, 平均色支配率 13.7%, 平均回転回数 126.9
初期上面 6、初期位置 (1, 3): 試合数 460, 勝率 79.6%, 平均色支配率 13.7%, 平均回転回数 126.0
初期上面 6、初期位置 (1, 4): 試合数 463, 勝率 81.9%, 平均色支配率 13.8%, 平均回転回数 130.5
初期上面 6、初期位置 (1, 5): 試合数 459, 勝率 69.3%, 平均色支配率 13.9%, 平均回転回数 128.7
初期上面 6、初期位置 (2, 0): 試合数 445, 勝率 43.1%, 平均色支配率 11.2%, 平均回転回数 107.5
初期上面 6、初期位置 (2, 1): 試合数 496, 勝率 56.0%, 平均色支配率 10.8%, 平均回転回数 127.1
初期上面 6、初期位置 (2, 2): 試合数 438, 勝率 56.2%, 平均色支配率 10.9%, 平均回転回数 126.1
初期上面 6、初期位置 (2, 3): 試合数 455, 勝率 61.3%, 平均色支配率 11.0%, 平均回転回数 127.1
初期上面 6、初期位置 (2, 4): 試合数 461, 勝率 51.8%, 平均色支配率 11.1%, 平均回転回数 127.9
初期上面 6、初期位置 (2, 5): 試合数 472, 勝率 53.6%, 平均色支配率 11.1%, 平均回転回数 127.9
初期上面 6、初期位置 (3, 0): 試合数 442, 勝率 29.9%, 平均色支配率 9.0%, 平均回転回数 102.2
初期上面 6、初期位置 (3, 1): 試合数 470, 勝率 36.6%, 平均色支配率 8.1%, 平均回転回数 127.7
初期上面 6、初期位置 (3, 2): 試合数 463, 勝率 43.6%, 平均色支配率 8.2%, 平均回転回数 127.1
初期上面 6、初期位置 (3, 3): 試合数 447, 勝率 39.1%, 平均色支配率 8.2%, 平均回転回数 125.4
初期上面 6、初期位置 (3, 4): 試合数 500, 勝率 39.2%, 平均色支配率 8.2%, 平均回転回数 128.9
初期上面 6、初期位置 (3, 5): 試合数 425, 勝率 31.5%, 平均色支配率 8.3%, 平均回転回数 127.2
初期上面 6、初期位置 (4, 0): 試合数 472, 勝率 13.3%, 平均色支配率 6.6%, 平均回転回数 108.9
初期上面 6、初期位置 (4, 1): 試合数 481, 勝率 21.6%, 平均色支配率 5.5%, 平均回転回数 128.2
初期上面 6、初期位置 (4, 2): 試合数 472, 勝率 20.1%, 平均色支配率 5.5%, 平均回転回数 125.6
初期上面 6、初期位置 (4, 3): 試合数 479, 勝率 20.0%, 平均色支配率 5.6%, 平均回転回数 125.9
初期上面 6、初期位置 (4, 4): 試合数 451, 勝率 17.1%, 平均色支配率 5.7%, 平均回転回数 127.4
初期上面 6、初期位置 (4, 5): 試合数 455, 勝率 18.9%, 平均色支配率 5.6%, 平均回転回数 128.5
初期上面 6、初期位置 (5, 0): 試合数 434, 勝率 4.1%, 平均色支配率 4.0%, 平均回転回数 110.1
初期上面 6、初期位置 (5, 1): 試合数 494, 勝率 0.0%, 平均色支配率 2.6%, 平均回転回数 127.0
初期上面 6、初期位置 (5, 2): 試合数 524, 勝率 0.4%, 平均色支配率 2.7%, 平均回転回数 125.9
初期上面 6、初期位置 (5, 3): 試合数 462, 勝率 0.0%, 平均色支配率 2.6%, 平均回転回数 129.6
初期上面 6、初期位置 (5, 4): 試合数 470, 勝率 0.0%, 平均色支配率 2.7%, 平均回転回数 127.4
初期上面 6、初期位置 (5, 5): 試合数 423, 勝率 0.0%, 平均色支配率 2.8%, 平均回転回数 129.9